# Black-box optimization exercise

## Introduction

This is part of the capstone project for the Professional Certificate in Machine Learning and AI from Imperial College Business School.

The goal is the optimization of 8 black-box functions. For each of the functions 10 initial data points are given and each week I must submit a new data point to evaluate. This simulates a costly process of actual evaluation.

All variables range from 0 to 1.

This notebook will document the whole process. It is structured in a way that follows, function by function and week by week, the thought process and techniques used.

This kind of problem is the textbook example of usage of Bayesian optimization. 

In this kind of problem the most widely used surrogate function is a Gaussian process because, among other things, it already comes with uncertainty measures embedded. I will be testing our different kernels and acquisition functions depending on the case at hand. The litterature suggests applying. Automatic relevance determination (ARD) to the kernels which allows for differnet "length scale" for each dimension. (https://www.pnas.org/doi/10.1073/pnas.1912342117)

## Overview

The challenge provided a certain number of initial points for all 8 functions and then, after each submission, files containing the results a provided.

In this notebook, a dataframe called results_df contains all such results, however they are loaded progressively week by week into variables called fx_inputs and fx_outputs, so as to guarantee that progressive data acquisition can be reproduced throught this notebook. It is therefore very important that all cells are executed in order.

### Environment preparation

In [ ]:
# Environment preparation, all libs imports

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# !pip install pydot
from IPython.display import Image, display
import os
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel as C, Matern
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import NearestNeighbors
from scipy.optimize import minimize
from scipy.spatial.distance import cdist
from scipy.stats import qmc
from scipy.stats import norm
from skopt import forest_minimize
from mpl_toolkits.mplot3d import Axes3D
import scipy.stats as stats
import ast, re
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import NearestNeighbors
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers, models

In [ ]:
# Environment preparation, populating `results_df`, which is a dataframe with all the results known for all functions

results_df = pd.DataFrame(columns=['week', 'function', 'known_best', 'provided_input', 'output', 'submission_improved'])

# Load initial values
for X in range(1, 9):
    inputs_path = f"initial_data/function_{X}/initial_inputs.npy"
    outputs_path = f"initial_data/function_{X}/initial_outputs.npy"
    
    globals()[f"f{X}_inputs"] = np.load(inputs_path)
    globals()[f"f{X}_outputs"] = np.load(outputs_path)

    results_df.loc[len(results_df)] = [0, X, np.max(globals()[f"f{X}_outputs"]), globals()[f"f{X}_inputs"][np.argmax(globals()[f"f{X}_outputs"])], np.nan, False]

# Load results dataframe with each weeks inputs and outputs
#for X in range(1,3):
inputs_path = f"submissions/w5_inputs.txt"
outputs_path = f"submissions/w5_outputs.txt"

#Inputs
with open(inputs_path, 'r') as file:
    content = file.read()

# Split out the separate sets
raw_sets = re.split(r'\]\s*\[', content.strip())
raw_sets = [s.strip('[]') for s in raw_sets]

# Parse each set into lists of numpy arrays
all_sets = []
for s in raw_sets:
    arrays_raw = re.findall(r'array\((.*?)\)', s, re.DOTALL)
    instance_set = [np.array(eval(a)) for a in arrays_raw]
    all_sets.append(instance_set)

# Transpose structure: arrays[i] has i-th array of each set
max_len = max(len(s) for s in all_sets)
arrays = []
for i in range(max_len):
    group = []
    for n, s in enumerate(all_sets, start=1):
        if i < len(s):
            #week number, function, best_output, provided_input, output, submission_improved
            results_df.loc[len(results_df)] = [n, i+1, np.nan, s[i], np.nan, False]
    
#Outputs
with open(outputs_path, 'r') as file:
    for line_number, line in enumerate(file, start=1):
        content = line.strip()

        # Replace 'np.float64(' and ')' so the string only has floats, then use ast.literal_eval for safety
        content_clean = content.replace('np.float64(', '').replace(')', '')
        float_list = ast.literal_eval(f'[{content_clean}]')

        outputs = np.array(float_list, dtype=np.float64)

        # for each function
        for i in range(0,8):
            #Determine if the submission if best than the known best previous week (hence the -1)
            row = results_df[(results_df['week'] == line_number-1) & (results_df['function'] == i+1)]
            known_best = row['known_best'].values[0]
        
            improved = (outputs[0,i] > known_best)
            if improved:
                known_best = outputs[0,i]
            # populate the dataframe
            condition = (results_df['week'] == line_number) & (results_df['function'] == i+1)
            results_df.loc[condition, 'submission_improved'] = improved
            results_df.loc[condition, 'known_best'] = known_best
            results_df.loc[condition, 'output'] = outputs[0,i]

In [ ]:
# Environment preparation, definition of some common functions



### Observation of weekly results

In [ ]:
results_df.sort_values(by=['function', 'week'])

results_df[results_df['week'] == 1]

In [ ]:
results_df[results_df['week'] == 2]

In [ ]:
results_df[results_df['week'] == 3]


In [ ]:
results_df[results_df['week'] == 4]

In [ ]:
results_df[results_df['week'] == 5]

## Function 1

This function is a function with a 1D output and a 2D input. 

This is the description of the function: Detect likely contamination sources in a two-dimensional area, such as a radiation field, where only proximity yields a non-zero reading. The system uses Bayesian optimisation to tune detection parameters and reliably identify both strong and weak sources.

### Week 1

This is a 2D function, so we can start by plotting the known datapoints

In [ ]:
x1 = f1_inputs[:, 0]
x2 = f1_inputs[:, 1]
y = f1_outputs

# Put data into a DataFrame
df = pd.DataFrame({
    "x1": x1,
    "x2": x2,
    "y": y
})

# Scatter plot with color encoding the output
plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x="x1", y="x2", hue="y", palette="coolwarm", s=60, legend=False)
plt.title("f1: Output as function of two inputs")
plt.xlabel("Input 1")
plt.ylabel("Input 2")
plt.xlim(0, 1)
plt.ylim(0, 1)
norm = plt.Normalize(vmin=y.min(), vmax=y.max())
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
plt.colorbar(sm, ax=plt.gca())
plt.show()


print(f"Evaluations: {len(f1_inputs)}")
print(f"Inputs shape: {f1_inputs.shape}")
print(f"Inputs range: [{np.min(f1_inputs):.4f}, {np.max(f1_inputs):.4f}]")
print(f"Performance range: [{np.min(f1_outputs):.4e}, {np.max(f1_outputs):.4e}]")
print(f"Current best: {np.max(f1_outputs):.4e}")
print(f"Inputs producing current best: {f1_inputs[np.argmax(f1_outputs)]}")

Most values are rather on the lower end, which is normal because radiation will be low outside the contaminated area. Peaks are expected to be localized.

We can try a Matern kernel in a Gaussian process regressor to consider that data is not smooth, with a smart selection of nu. If nu tends to infinity, then this behaves as a RBF kernel (reference: https://andrewcharlesjones.github.io/journal/matern-kernels.html). We will initially try with a nu of 1.5

In [ ]:
kernel = Matern(length_scale=[0.1, 0.1], length_scale_bounds=(0.05, 0.5), nu=1.5)
gp = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-6, #modelling noise     
    normalize_y=True,
    random_state=42
)
gp.fit(f1_inputs, f1_outputs)

x1 = np.linspace(0, 1, 100)
x2 = np.linspace(0, 1, 100)
X1, X2 = np.meshgrid(x1, x2)
X_grid = np.column_stack([X1.ravel(), X2.ravel()])

y_mean, y_std = gp.predict(X_grid, return_std=True)
Y_mean = y_mean.reshape(X1.shape)
Y_std = y_std.reshape(X1.shape)

# Plot mean
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.contourf(X1, X2, Y_mean, cmap='coolwarm', levels=50)
plt.scatter(f1_inputs[:,0], f1_inputs[:,1], c=f1_outputs, cmap='coolwarm', edgecolor='k')
plt.colorbar(label='Mean prediction')
plt.title("GP Mean Prediction (Matern Kernel)")
plt.xlabel("Input 1")
plt.ylabel("Input 2")

# Plot uncertainty
plt.subplot(1,2,2)
plt.contourf(X1, X2, Y_std, cmap='viridis', levels=50)
plt.colorbar(label='Standard deviation (uncertainty)')
plt.scatter(f1_inputs[:,0], f1_inputs[:,1], c='k', s=20)
plt.title("GP Uncertainty")
plt.xlabel("Input 1")
plt.ylabel("Input 2")

plt.tight_layout()
plt.show()

I am getting warnings about the optimal values being to close to the bounds provided, so let's try again with new values.

In [ ]:

kernel = Matern(length_scale=[0.1, 0.1], length_scale_bounds=(1e-2, 1e5), nu=1.5)
gp = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-6, #modelling noise     
    normalize_y=True,
    random_state=42
)
gp.fit(f1_inputs, f1_outputs)

x1 = np.linspace(0, 1, 100)
x2 = np.linspace(0, 1, 100)
X1, X2 = np.meshgrid(x1, x2)
X_grid = np.column_stack([X1.ravel(), X2.ravel()])

y_mean, y_std = gp.predict(X_grid, return_std=True)
Y_mean = y_mean.reshape(X1.shape)
Y_std = y_std.reshape(X1.shape)

# Plot mean
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.contourf(X1, X2, Y_mean, cmap='coolwarm', levels=50)
plt.scatter(f1_inputs[:,0], f1_inputs[:,1], c=f1_outputs, cmap='coolwarm', edgecolor='k')
plt.colorbar(label='Mean prediction')
plt.title("GP Mean Prediction (Matern Kernel)")
plt.xlabel("Input 1")
plt.ylabel("Input 2")

# Plot uncertainty
plt.subplot(1,2,2)
plt.contourf(X1, X2, Y_std, cmap='viridis', levels=50)
plt.colorbar(label='Standard deviation (uncertainty)')
plt.scatter(f1_inputs[:,0], f1_inputs[:,1], c='k', s=20)
plt.title("GP Uncertainty")
plt.xlabel("Input 1")
plt.ylabel("Input 2")

plt.tight_layout()
plt.show()

As this is a problem that must encourage exploration, let's use a UCB acquisition function, with a choice of k = 3.

In [ ]:
kappa = 3.0
UCB = Y_mean + kappa * Y_std  # shape = (grid_size, grid_size)

plt.figure(figsize=(6,5))
plt.contourf(X1, X2, UCB, cmap='plasma', levels=50)
plt.scatter(f1_inputs[:,0], f1_inputs[:,1], c='k', s=50, label='Observed points')
plt.colorbar(label='UCB acquisition value')
plt.title(f"UCB Acquisition Function (kappa={kappa})")
plt.xlabel("Input 1")
plt.ylabel("Input 2")
plt.legend()
plt.show()

In [ ]:
max_idx = np.argmax(UCB)  # index of the maximum UCB
next_point = X_grid[max_idx]  # coordinates in input space

print(f"Current best: {np.max(f1_outputs):.4e}")
print(f"Inputs producing current best: {f1_inputs[np.argmax(f1_outputs)]}")
print("Next point to sample based on UCB:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

# With the other bounds the suggested submission was 0.78787879 0.7979798

Submission for week 1: 0.373737-0.232323

### Week 2

In [ ]:
#Load points to inputs and outputs from previous week
function = 1
week = 1 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f1_inputs = np.vstack([f1_inputs, row['provided_input'].values[0]])
f1_outputs = np.append(f1_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f1_outputs):.4e}")
print(f"Last week's results: {row['output'].values[0]:.4e}")


Let's try the same thing but adjusting the surrogate function so that the importance is given to both parameters. UCB as acquisition function favors exploration, which is what we want at this point.

In [ ]:

kernel = C(1.0, (1e-5, 1e5)) * Matern(length_scale=[0.05, 0.05], length_scale_bounds=(0.01, 0.3), nu=2.5) + WhiteKernel(noise_level=1e-8, noise_level_bounds=(1e-12, 1e2))
gp = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=50,
    alpha=1e-6, #modelling noise     
    normalize_y=True,
    random_state=42
)
gp.fit(f1_inputs, f1_outputs)

x1 = np.linspace(0, 1, 100)
x2 = np.linspace(0, 1, 100)
X1, X2 = np.meshgrid(x1, x2)
X_grid = np.column_stack([X1.ravel(), X2.ravel()])

y_mean, y_std = gp.predict(X_grid, return_std=True)
Y_mean = y_mean.reshape(X1.shape)
Y_std = y_std.reshape(X1.shape)

# Plot mean
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.contourf(X1, X2, Y_mean, cmap='coolwarm', levels=50)
plt.scatter(f1_inputs[:,0], f1_inputs[:,1], c=f1_outputs, cmap='coolwarm', edgecolor='k')
plt.colorbar(label='Mean prediction')
plt.title("GP Mean Prediction (Matern Kernel)")
plt.xlabel("Input 1")
plt.ylabel("Input 2")

# Plot uncertainty
plt.subplot(1,2,2)
plt.contourf(X1, X2, Y_std, cmap='viridis', levels=50)
plt.colorbar(label='Standard deviation (uncertainty)')
plt.scatter(f1_inputs[:,0], f1_inputs[:,1], c='k', s=20)
plt.title("GP Uncertainty")
plt.xlabel("Input 1")
plt.ylabel("Input 2")

plt.tight_layout()
plt.show()

In [ ]:
kappa = 3.0
UCB = Y_mean + kappa * Y_std  # shape = (grid_size, grid_size)

plt.figure(figsize=(6,5))
plt.contourf(X1, X2, UCB, cmap='plasma', levels=50)
plt.scatter(f1_inputs[:,0], f1_inputs[:,1], c='k', s=50, label='Observed points')
plt.colorbar(label='UCB acquisition value')
plt.title(f"UCB Acquisition Function (kappa={kappa})")
plt.xlabel("Input 1")
plt.ylabel("Input 2")
plt.legend()
plt.show()

In [ ]:
max_idx = np.argmax(UCB)  # index of the maximum UCB
next_point = X_grid[max_idx]  # coordinates in input space

print(f"Current best: {np.max(f1_outputs):.4e}")
print(f"Inputs producing current best: {f1_inputs[np.argmax(f1_outputs)]}")
print("Next point to sample based on UCB:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 2: 0.989899-0.767677

### Week 3

In [ ]:
#Load points to inputs and outputs from previous week
function = 1
week = 2 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f1_inputs = np.vstack([f1_inputs, row['provided_input'].values[0]])
f1_outputs = np.append(f1_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f1_outputs):.4e}")
print(f"Last week's results: {row['output'].values[0]:.4e}")
print('--')
print(f"Inputs shape: {f1_inputs.shape}")
print(f"Inputs range: [{np.min(f1_inputs):.4f}, {np.max(f1_inputs):.4f}]")
print(f"Performance range: [{np.min(f1_outputs):.4e}, {np.max(f1_outputs):.4e}]")

In [ ]:
x1 = f1_inputs[:, 0]
x2 = f1_inputs[:, 1]
y = f1_outputs

# Put data into a DataFrame
df = pd.DataFrame({
    "x1": x1,
    "x2": x2,
    "y": y
})

# Scatter plot with color encoding the output
plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x="x1", y="x2", hue="y", palette="coolwarm", s=60, legend=False)
plt.title("f1: Output as function of two inputs")
plt.xlabel("Input 1")
plt.ylabel("Input 2")
plt.xlim(0, 1)
plt.ylim(0, 1)
norm = plt.Normalize(vmin=y.min(), vmax=y.max())
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
plt.colorbar(sm, ax=plt.gca())
plt.show()


print(f"Evaluations: {len(f1_inputs)}")
print(f"Inputs shape: {f1_inputs.shape}")
print(f"Inputs range: [{np.min(f1_inputs):.4f}, {np.max(f1_inputs):.4f}]")
print(f"Performance range: [{np.min(f1_outputs):.4e}, {np.max(f1_outputs):.4e}]")
print(f"Current best: {np.max(f1_outputs):.4e}")
print(f"Inputs producing current best: {f1_inputs[np.argmax(f1_outputs)]}")

We haven't had any improvement for this function so far. It is known that values only peak close to a contamination source. One has already been identified (blue dot above), the goal is to determine if other exist in the 2D plane.

In preparation for more complex cases, I am exploring forest_minimize from skopt which efficiently maximizes the value of the acquisition function. This will help also with high-dimensional cases. I am using a L-BFGS-B (Limited-memory Broyden–Fletcher–Goldfarb–Shanno with Bounds), which according to my limited research is optimal for bounded sets of input values.

In [ ]:
# Define kernel: Matern with nu=2.5 for sharp peaks
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[0.2]*2, nu=1.5)

# Instantiate GP with small noise level
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-5, normalize_y=True)

# Fit the GP model
gp.fit(f1_inputs, f1_outputs)
print(gp.kernel_)


# Function to optimize acquisition: Expected Improvement
def acquisitionf(x):
    x = np.array(x).reshape(1, -1)
    mean, std = gp.predict(x, return_std=True)
    # Current best observation
    best_y = np.max(f1_outputs)
    # Calculate EI
    z = (mean - best_y) / (std + 1e-9)
    ei = (mean - best_y) * norm.cdf(z) + std * norm.pdf(z)
    return -ei.item()  

# Robust acquisition maximization via multi-start local optimization
bounds = [(0, 1), (0, 1)]
num_restarts = 1500 # I have incremented this until the estimation did no longer move (fixed random seed)
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, 2)
    res = minimize(acquisitionf, x0, bounds=bounds, method='L-BFGS-B')
    results.append(res)

print(f"Current best: {np.max(f1_outputs):.4e}")
print(f"Inputs producing current best: {f1_inputs[np.argmax(f1_outputs)]}")

# Select best solution from restarts
next_point = min(results, key=lambda r: r.fun).x
print("Next sample point (local optimizer):", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 3: 0.405019-0.162145

### Week 4

In [ ]:
#Load points to inputs and outputs from previous week
function = 1
week = 3 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f1_inputs = np.vstack([f1_inputs, row['provided_input'].values[0]])
f1_outputs = np.append(f1_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f1_outputs):.4e}")
print(f"Last week's results: {row['output'].values[0]:.4e}")
print('--')
print(f"Inputs shape: {f1_inputs.shape}")
print(f"Inputs range: [{np.min(f1_inputs):.4f}, {np.max(f1_inputs):.4f}]")
print(f"Performance range: [{np.min(f1_outputs):.4e}, {np.max(f1_outputs):.4e}]")

No improvement has been possible so far. The maximum value is part of the initial values.

In [ ]:
x1 = f1_inputs[:, 0]
x2 = f1_inputs[:, 1]
y = f1_outputs

# Put data into a DataFrame
df = pd.DataFrame({
    "x1": x1,
    "x2": x2,
    "y": y
})

# Scatter plot with color encoding the output
plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x="x1", y="x2", hue="y", palette="coolwarm", s=60, legend=False)
plt.title("f1: Output as function of two inputs")
plt.xlabel("Input 1")
plt.ylabel("Input 2")
plt.xlim(0, 1)
plt.ylim(0, 1)
norm = plt.Normalize(vmin=y.min(), vmax=y.max())
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
plt.colorbar(sm, ax=plt.gca())
plt.show()


print(f"Evaluations: {len(f1_inputs)}")
print(f"Inputs shape: {f1_inputs.shape}")
print(f"Inputs range: [{np.min(f1_inputs):.4f}, {np.max(f1_inputs):.4f}]")
print(f"Performance range: [{np.min(f1_outputs):.4e}, {np.max(f1_outputs):.4e}]")
print(f"Current best: {np.max(f1_outputs):.4e}")
print(f"Inputs producing current best: {f1_inputs[np.argmax(f1_outputs)]}")

In [ ]:
# Define kernel: Matern with nu=2.5 for sharp peaks
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[0.2]*2, nu=1.5)

# Instantiate GP with small noise level
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-5, normalize_y=True)

# Fit the GP model
gp.fit(f1_inputs, f1_outputs)
print(gp.kernel_)


# Function to optimize acquisition: Expected Improvement
def acquisitionf(x):
    x = np.array(x).reshape(1, -1)
    mean, std = gp.predict(x, return_std=True)
    # Current best observation
    best_y = np.max(f1_outputs)
    # Calculate EI
    z = (mean - best_y) / (std + 1e-9)
    ei = (mean - best_y) * norm.cdf(z) + std * norm.pdf(z)
    return -ei.item()  

# Robust acquisition maximization via multi-start local optimization
bounds = [(0, 1), (0, 1)]
num_restarts = 150 
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, 2)
    res = minimize(acquisitionf, x0, bounds=bounds, method='L-BFGS-B')
    results.append(res)

print(f"Current best: {np.max(f1_outputs):.4e}")
print(f"Inputs producing current best: {f1_inputs[np.argmax(f1_outputs)]}")

# Select best solution from restarts
next_point_ei = min(results, key=lambda r: r.fun).x
print("Next sample point (ei):", next_point_ei)


def negative_ucb(x, gp, kappa):
    mu, sigma = gp.predict(x.reshape(1, -1), return_std=True)
    return -(mu[0] + kappa * sigma[0])

dims = 2
bounds = [(0, 1) for _ in range(dims)]
num_restarts = 50
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, dims)
    #choosing kappa 2.7 
    result = minimize(negative_ucb, x0=np.random.uniform(0, 1, dims), 
                 args=(gp, 1.7), bounds=bounds, method='L-BFGS-B')
    if result.success:
        results.append(result)
    
# Select best solution from restarts
next_point_ucb = min(results, key=lambda r: r.fun).x
print("Next sample point (ucb):", next_point_ucb)


print("-".join(f"{x:.6f}" for x in next_point_ucb.flatten()))

I choose the UCB value.

Sample for week 4: 0.871653-0.585572

### Week 5

In [ ]:
#Load points to inputs and outputs from previous week
function = 1
week = 4 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f1_inputs = np.vstack([f1_inputs, row['provided_input'].values[0]])
f1_outputs = np.append(f1_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f1_outputs):.4e}")
print(f"Last week's results: {row['output'].values[0]:.4e}")
print('--')
print(f"Inputs shape: {f1_inputs.shape}")
print(f"Inputs range: [{np.min(f1_inputs):.4f}, {np.max(f1_inputs):.4f}]")
print(f"Performance range: [{np.min(f1_outputs):.4e}, {np.max(f1_outputs):.4e}]")

Still not found a non-zero reading. The maximum is close to 0. The description of the function cleams that only proximity to a contamination source yield a non-zero reading.

This week I am choosing to sample in the middle of the square:

In [ ]:
chosen_point = np.array([0.5, 0.5])
print("-".join(f"{x:.6f}" for x in chosen_point.flatten()))

Submissison for week 5: 0.500000-0.500000

### Week 6

In [ ]:
#Load points to inputs and outputs from previous week
function = 1
week = 5 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f1_inputs = np.vstack([f1_inputs, row['provided_input'].values[0]])
f1_outputs = np.append(f1_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f1_outputs):.4e}")
print(f"Last week's results: {row['output'].values[0]:.4e}")
print('--')
print(f"Inputs shape: {f1_inputs.shape}")
print(f"Inputs range: [{np.min(f1_inputs):.4f}, {np.max(f1_inputs):.4f}]")
print(f"Performance range: [{np.min(f1_outputs):.4e}, {np.max(f1_outputs):.4e}]")

Sampling the middle has yielded the best value so far, slightly positive. 

In [ ]:
x1 = f1_inputs[:, 0]
x2 = f1_inputs[:, 1]
y = np.log(f1_outputs)

# Put data into a DataFrame
df = pd.DataFrame({
    "x1": x1,
    "x2": x2,
    "y": y
})

# Scatter plot with color encoding the output
plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x="x1", y="x2", hue="y", palette="coolwarm", s=60, legend=False)
plt.title("f1: Log(Output) as function of two inputs")
plt.xlabel("Input 1")
plt.ylabel("Input 2")
plt.xlim(0, 1)
plt.ylim(0, 1)
norm = plt.Normalize(vmin=y.min(), vmax=y.max())
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
plt.colorbar(sm, ax=plt.gca())
plt.show()


print(f"Evaluations: {len(f1_inputs)}")
print(f"Inputs shape: {f1_inputs.shape}")
print(f"Inputs range: [{np.min(f1_inputs):.4f}, {np.max(f1_inputs):.4f}]")
print(f"Performance range: [{np.min(f1_outputs):.4e}, {np.max(f1_outputs):.4e}]")
print(f"Current best: {np.max(f1_outputs):.4e}")
print(f"Inputs producing current best: {f1_inputs[np.argmax(f1_outputs)]}")

I am once again sampling a point with little information selected manually so that I can use those to better approximate the underlying function.

In [ ]:
chosen_point = np.array([0.3, 0.2])
print("-".join(f"{x:.6f}" for x in chosen_point.flatten()))

Submission for week 6: 0.300000-0.200000

### Week 7

In [ ]:
#Load points to inputs and outputs from previous week
function = 1
week = 6 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f1_inputs = np.vstack([f1_inputs, row['provided_input'].values[0]])
f1_outputs = np.append(f1_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f1_outputs):.4e}")
print(f"Last week's results: {row['output'].values[0]:.4e}")
print('--')
print(f"Inputs shape: {f1_inputs.shape}")
print(f"Inputs range: [{np.min(f1_inputs):.4f}, {np.max(f1_inputs):.4f}]")
print(f"Performance range: [{np.min(f1_outputs):.4e}, {np.max(f1_outputs):.4e}]")

This week I am choosing to use another Gaussian process with an UCB aquisition function, but I am going to approximate log(y) instead of y.

In [ ]:
# code...

## Function 2

This function is a function with a 1D output and a 2D input. 

This is the description of the function: Imagine a black box, or a mystery ML model, that takes two numbers as input and returns a log-likelihood score. Your goal is to maximise that score, but each output is noisy, and depending on where you start, you might get stuck in a local optimum. 

To tackle this, you use Bayesian optimisation, which selects the next inputs based on what it has learned so far. It balances exploration with exploitation, making it well suited to noisy outputs and complex functions with many local peaks.

### Week 1

This is a 2D function, so we can start by plotting the known datapoints

In [ ]:
x1 = f2_inputs[:, 0]
x2 = f2_inputs[:, 1]
y = f2_outputs

# Put data into a DataFrame
df = pd.DataFrame({
    "x1": x1,
    "x2": x2,
    "y": y
})

# Scatter plot with color encoding the output
plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x="x1", y="x2", hue="y", palette="coolwarm", s=60, legend=False)
plt.title("f2: Output as function of two inputs")
plt.xlabel("Input 1")
plt.ylabel("Input 2")
plt.xlim(0, 1)
plt.ylim(0, 1)
norm = plt.Normalize(vmin=y.min(), vmax=y.max())
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
plt.colorbar(sm, ax=plt.gca())
plt.show()


print(f"Evaluations: {len(f2_inputs)}")
print(f"Inputs shape: {f2_inputs.shape}")
print(f"Inputs range: [{np.min(f2_inputs):.4f}, {np.max(f2_inputs):.4f}]")
print(f"Performance range: [{np.min(f2_outputs):.4f}, {np.max(f2_outputs):.4f}]")
print(f"Current best: {np.max(f2_outputs):.4f}")
print(f"Inputs producing current best: {f2_inputs[np.argmax(f2_outputs)]}")



Data is noisy.

We can try a RBF kernel combined with a WhiteKernel for noise in a Gaussian process regressor to consider that data is smooth as expected in a log likelihood.

In [ ]:
kernel = RBF(length_scale=0.2, length_scale_bounds=(1e-10,1)) + WhiteKernel(noise_level=1e-9, noise_level_bounds=(1e-12, 1e-3))
gp = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-8, #modelling noise     
    normalize_y=True,
    random_state=42
)
gp.fit(f2_inputs, f2_outputs)

x1 = np.linspace(0, 1, 100)
x2 = np.linspace(0, 1, 100)
X1, X2 = np.meshgrid(x1, x2)
X_grid = np.column_stack([X1.ravel(), X2.ravel()])

y_mean, y_std = gp.predict(X_grid, return_std=True)
Y_mean = y_mean.reshape(X1.shape)
Y_std = y_std.reshape(X1.shape)

# Plot mean
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.contourf(X1, X2, Y_mean, cmap='coolwarm', levels=50)
plt.scatter(f2_inputs[:,0], f2_inputs[:,1], c=f2_outputs, cmap='coolwarm', edgecolor='k')
plt.colorbar(label='Mean prediction')
plt.title("GP Mean Prediction (Matern Kernel)")
plt.xlabel("Input 1")
plt.ylabel("Input 2")

# Plot uncertainty
plt.subplot(1,2,2)
plt.contourf(X1, X2, Y_std, cmap='viridis', levels=50)
plt.colorbar(label='Standard deviation (uncertainty)')
plt.scatter(f2_inputs[:,0], f2_inputs[:,1], c='k', s=20)
plt.title("GP Uncertainty")
plt.xlabel("Input 1")
plt.ylabel("Input 2")

plt.tight_layout()
plt.show()

Let's use a UCB acquisition function, with a choice of k = 2.5.

In [ ]:
kappa = 2.5
UCB = Y_mean + kappa * Y_std  # shape = (grid_size, grid_size)

plt.figure(figsize=(6,5))
plt.contourf(X1, X2, UCB, cmap='plasma', levels=50)
plt.scatter(f2_inputs[:,0], f2_inputs[:,1], c='k', s=50, label='Observed points')
plt.colorbar(label='UCB acquisition value')
plt.title(f"UCB Acquisition Function (kappa={kappa})")
plt.xlabel("Input 1")
plt.ylabel("Input 2")
plt.legend()
plt.show()

In [ ]:
max_idx = np.argmax(UCB)  # index of the maximum UCB
next_point = X_grid[max_idx]  # coordinates in input space

print(f"Current best: {np.max(f2_outputs):.4f}")
print(f"Inputs producing current best: {f2_inputs[np.argmax(f2_outputs)]}")
print("Next point to sample based on UCB:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 1: 0.818182-0.979798


### Week 2

In [ ]:
#Load points to inputs and outputs from previous week
function = 2
week = 1 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f2_inputs = np.vstack([f2_inputs, row['provided_input'].values[0]])
f2_outputs = np.append(f2_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f2_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")

In [ ]:
print(f"Correlation between inputs: {np.corrcoef(f2_inputs, rowvar=False)}")

print('Means:', f2_inputs.mean(axis=0))
print('Stds:', f2_inputs.std(axis=0))
print('Mins:', f2_inputs.min(axis=0))
print('Maxs:', f2_inputs.max(axis=0))
print('Std/Mean Ratios:', f2_inputs.std(axis=0) / (f2_inputs.mean(axis=0) + 1e-12))

Let's try again a similar Gaussian process and plot the results with the new datapoint in hand.

In [ ]:
kernel = RBF(length_scale=0.2, length_scale_bounds=(1e-10,1)) + WhiteKernel(noise_level=1e-9, noise_level_bounds=(1e-12, 1e-3))
gp = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-8, #modelling noise     
    normalize_y=True,
    random_state=42
)
gp.fit(f2_inputs, f2_outputs)

x1 = np.linspace(0, 1, 100)
x2 = np.linspace(0, 1, 100)
X1, X2 = np.meshgrid(x1, x2)
X_grid = np.column_stack([X1.ravel(), X2.ravel()])

y_mean, y_std = gp.predict(X_grid, return_std=True)
Y_mean = y_mean.reshape(X1.shape)
Y_std = y_std.reshape(X1.shape)

# Plot mean
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.contourf(X1, X2, Y_mean, cmap='coolwarm', levels=50)
plt.scatter(f2_inputs[:,0], f2_inputs[:,1], c=f2_outputs, cmap='coolwarm', edgecolor='k')
plt.colorbar(label='Mean prediction')
plt.title("GP Mean Prediction (Matern Kernel)")
plt.xlabel("Input 1")
plt.ylabel("Input 2")

# Plot uncertainty
plt.subplot(1,2,2)
plt.contourf(X1, X2, Y_std, cmap='viridis', levels=50)
plt.colorbar(label='Standard deviation (uncertainty)')
plt.scatter(f2_inputs[:,0], f2_inputs[:,1], c='k', s=20)
plt.title("GP Uncertainty")
plt.xlabel("Input 1")
plt.ylabel("Input 2")

plt.tight_layout()
plt.show()

kappa = 2.5
UCB = Y_mean + kappa * Y_std  # shape = (grid_size, grid_size)

plt.figure(figsize=(6,5))
plt.contourf(X1, X2, UCB, cmap='plasma', levels=50)
plt.scatter(f2_inputs[:,0], f2_inputs[:,1], c='k', s=50, label='Observed points')
plt.colorbar(label='UCB acquisition value')
plt.title(f"UCB Acquisition Function (kappa={kappa})")
plt.xlabel("Input 1")
plt.ylabel("Input 2")
plt.legend()
plt.show()

In [ ]:
max_idx = np.argmax(UCB)  # index of the maximum UCB
next_point = X_grid[max_idx]  # coordinates in input space

print(f"Inputs producing current best: {f2_inputs[np.argmax(f2_outputs)]}")
print("Next point to sample based on UCB:", next_point)

The acquisition function is basically suggesting to sample around already known points that are close to the maximum. As we are still on week 2, it is worth to use exploration before exploiting the known areas, especially since we are told that it is easy to get stuck at a local minimum.

Let's try a different surrogate function with a penalized UCB for getting close to the borders.

In [ ]:
kernel = C(1.0, (1e-2, 10.0)) * Matern(length_scale=[1.0, 1.0], length_scale_bounds=(0.01, 10.0), nu=2.5)
gp = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-8, #modelling noise     
    normalize_y=True,
    random_state=42
)
gp.fit(f2_inputs, f2_outputs)

print(gp.kernel_)

x1 = np.linspace(0, 1, 100)
x2 = np.linspace(0, 1, 100)
X1, X2 = np.meshgrid(x1, x2)
X_grid = np.column_stack([X1.ravel(), X2.ravel()])

print(X_grid.shape)

y_mean, y_std = gp.predict(X_grid, return_std=True)
Y_mean = y_mean.reshape(X1.shape)
Y_std = y_std.reshape(X1.shape)

# Plot mean
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.contourf(X1, X2, Y_mean, cmap='coolwarm', levels=50)
plt.scatter(f2_inputs[:,0], f2_inputs[:,1], c=f2_outputs, cmap='coolwarm', edgecolor='k')
plt.colorbar(label='Mean prediction')
plt.title("GP Mean Prediction (Matern Kernel)")
plt.xlabel("Input 1")
plt.ylabel("Input 2")

# Plot uncertainty
plt.subplot(1,2,2)
plt.contourf(X1, X2, Y_std, cmap='viridis', levels=50)
plt.colorbar(label='Standard deviation (uncertainty)')
plt.scatter(f2_inputs[:,0], f2_inputs[:,1], c='k', s=20)
plt.title("GP Uncertainty")
plt.xlabel("Input 1")
plt.ylabel("Input 2")

plt.tight_layout()
plt.show()

kappa = 1.5
UCB = Y_mean + kappa * Y_std  # shape = (grid_size, grid_size)


# Let's add a penalty for getting close to the borders
# Calculate distance to nearest boundary for each dimension (0 and 1 here)
min_boundary_distance = np.minimum(np.min(X_grid, axis=1), np.min(1 - X_grid, axis=1))

# Penalty factor: linearly scale with distance from boundary (0 at boundary, 1 at distance >= penalty_radius)
penalty_radius = 0.1
penalty_factor = np.clip(min_boundary_distance / penalty_radius, 0, 1)

# Apply penalty
UCB = UCB.reshape(100, 100)
penalty_factor = penalty_factor.reshape(100, 100)
UCB_penalized = UCB * penalty_factor

plt.figure(figsize=(6,5))
plt.contourf(X1, X2, UCB_penalized, cmap='plasma', levels=50)
plt.scatter(f2_inputs[:,0], f2_inputs[:,1], c='k', s=50, label='Observed points')
plt.colorbar(label='UCB acquisition value')
plt.title(f"UCB Acquisition Function (kappa={kappa})")
plt.xlabel("Input 1")
plt.ylabel("Input 2")
plt.legend()
plt.show()

print(gp.kernel_)


In [ ]:
max_idx = np.argmax(UCB_penalized)  # index of the maximum UCB
next_point = X_grid[max_idx]  # coordinates in input space

print(f"Current best: {np.max(f2_outputs):.4f}")
print(f"Inputs producing current best: {f2_inputs[np.argmax(f2_outputs)]}")
print("Next point to sample based on UCB:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 2: 0.707071-0.101010

### Week 3

In [ ]:
#Load points to inputs and outputs from previous week
function = 2
week = 2 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f2_inputs = np.vstack([f2_inputs, row['provided_input'].values[0]])
f2_outputs = np.append(f2_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f2_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f2_inputs.shape}")
print(f"Inputs range: [{np.min(f2_inputs):.4f}, {np.max(f2_inputs):.4f}]")
print(f"Performance range: [{np.min(f2_outputs):.4e}, {np.max(f2_outputs):.4e}]")

In [ ]:
x1 = f2_inputs[:, 0]
x2 = f2_inputs[:, 1]
y = f2_outputs

# Put data into a DataFrame
df = pd.DataFrame({
    "x1": x1,
    "x2": x2,
    "y": y
})

# Scatter plot with color encoding the output
plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x="x1", y="x2", hue="y", palette="coolwarm", s=60, legend=False)
plt.title("f2: Output as function of two inputs")
plt.xlabel("Input 1")
plt.ylabel("Input 2")
plt.xlim(0, 1)
plt.ylim(0, 1)
norm = plt.Normalize(vmin=y.min(), vmax=y.max())
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
plt.colorbar(sm, ax=plt.gca())
plt.show()


print(f"Evaluations: {len(f2_inputs)}")
print(f"Inputs shape: {f2_inputs.shape}")
print(f"Inputs range: [{np.min(f2_inputs):.4f}, {np.max(f2_inputs):.4f}]")
print(f"Performance range: [{np.min(f2_outputs):.4f}, {np.max(f2_outputs):.4f}]")
print(f"Current best: {np.max(f2_outputs):.4f}")
print(f"Inputs producing current best: {f2_inputs[np.argmax(f2_outputs)]}")

Even though the performance did not improve this week, we found potential for another local maximum.

In [ ]:
# Define kernel: Matern with nu=2.5 for sharp peaks
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[0.2, 0.2], nu=2.5)

# Instantiate GP with small noise level
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True)

# Fit the GP model
gp.fit(f2_inputs, f2_outputs)
print(gp.kernel_)


# Function to optimize acquisition: Expected Improvement
def acquisitionf(x):
    x = np.array(x).reshape(1, -1)
    mean, std = gp.predict(x, return_std=True)
    # Current best observation
    best_y = np.max(f1_outputs)
    # Calculate EI
    z = (mean - best_y) / (std + 1e-9)
    ei = (mean - best_y) * norm.cdf(z) + std * norm.pdf(z)
    return -ei.item()  

# Robust acquisition maximization via multi-start local optimization
bounds = [(0, 1), (0, 1)]
num_restarts = 1500 # I have incremented this until the estimation did no longer move (fixed random seed)
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, 2)
    res = minimize(acquisitionf, x0, bounds=bounds, method='L-BFGS-B')
    results.append(res)

print(f"Current best: {np.max(f2_outputs):.4f}")
print(f"Inputs producing current best: {f2_inputs[np.argmax(f2_outputs)]}")

# Select best solution from restarts
next_point = min(results, key=lambda r: r.fun).x
print("Next sample point (local optimizer):", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 3: 0.693605-0.805808

Which is along the vertical line that we found to seem to contain maxima.

### Week 4

In [ ]:
#Load points to inputs and outputs from previous week
function = 2
week = 3 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f2_inputs = np.vstack([f2_inputs, row['provided_input'].values[0]])
f2_outputs = np.append(f2_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f2_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f2_inputs.shape}")
print(f"Inputs range: [{np.min(f2_inputs):.4f}, {np.max(f2_inputs):.4f}]")
print(f"Performance range: [{np.min(f2_outputs):.4e}, {np.max(f2_outputs):.4e}]")

In [ ]:
x1 = f2_inputs[:, 0]
x2 = f2_inputs[:, 1]
y = f2_outputs

# Put data into a DataFrame
df = pd.DataFrame({
    "x1": x1,
    "x2": x2,
    "y": y
})

# Scatter plot with color encoding the output
plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x="x1", y="x2", hue="y", palette="coolwarm", s=60, legend=False)
plt.title("f2: Output as function of two inputs")
plt.xlabel("Input 1")
plt.ylabel("Input 2")
plt.xlim(0, 1)
plt.ylim(0, 1)
norm = plt.Normalize(vmin=y.min(), vmax=y.max())
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
plt.colorbar(sm, ax=plt.gca())
plt.show()


print(f"Evaluations: {len(f2_inputs)}")
print(f"Inputs shape: {f2_inputs.shape}")
print(f"Inputs range: [{np.min(f2_inputs):.4f}, {np.max(f2_inputs):.4f}]")
print(f"Performance range: [{np.min(f2_outputs):.4f}, {np.max(f2_outputs):.4f}]")
print(f"Current best: {np.max(f2_outputs):.4f}")
print(f"Inputs producing current best: {f2_inputs[np.argmax(f2_outputs)]}")

This confirms that there is a vertical line along which the function evaluates to the highest values.

In [ ]:
# Define kernel: Matern with nu=2.5 for sharp peaks
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[0.2, 0.2], nu=2.5)

# Instantiate GP with small noise level
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True)

# Fit the GP model
gp.fit(f2_inputs, f2_outputs)
print(gp.kernel_)


# Function to optimize acquisition: Expected Improvement
def acquisitionf(x):
    x = np.array(x).reshape(1, -1)
    mean, std = gp.predict(x, return_std=True)
    # Current best observation
    best_y = np.max(f1_outputs)
    # Calculate EI
    z = (mean - best_y) / (std + 1e-9)
    ei = (mean - best_y) * norm.cdf(z) + std * norm.pdf(z)
    return -ei.item()  

# Robust acquisition maximization via multi-start local optimization
bounds = [(0, 1), (0, 1)]
num_restarts = 1500 # I have incremented this until the estimation did no longer move (fixed random seed)
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, 2)
    res = minimize(acquisitionf, x0, bounds=bounds, method='L-BFGS-B')
    results.append(res)

print(f"Current best: {np.max(f2_outputs):.4f}")
print(f"Inputs producing current best: {f2_inputs[np.argmax(f2_outputs)]}")

# Select best solution from restarts
next_point = min(results, key=lambda r: r.fun).x
print("Next sample point (local optimizer):", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 4: 0.688772-0.849196

### Week 5

In [ ]:
#Load points to inputs and outputs from previous week
function = 2
week = 4 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f2_inputs = np.vstack([f2_inputs, row['provided_input'].values[0]])
f2_outputs = np.append(f2_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f2_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f2_inputs.shape}")
print(f"Inputs range: [{np.min(f2_inputs):.4f}, {np.max(f2_inputs):.4f}]")
print(f"Performance range: [{np.min(f2_outputs):.4e}, {np.max(f2_outputs):.4e}]")

The description of the function tells us to keep exploring bayesian optimisation, so I will continue using surrogate Gaussian processes for this week with tweaked hyperparameters and still some exploration bias.

In [ ]:
dims = 2

# Define kernel: Matern with nu=2.5 for sharp peaks
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[0.2, 0.2], nu=1.5)

# Instantiate GP with small noise level
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True)

# Fit the GP model
gp.fit(f2_inputs, f2_outputs)
print(gp.kernel_)

def negative_ucb(x, gp, kappa):
    mu, sigma = gp.predict(x.reshape(1, -1), return_std=True)
    return -(mu[0] + kappa * sigma[0])

bounds = [(0, 1) for _ in range(dims)]
num_restarts = 150
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, dims)
    #choosing kappa 1.5
    result = minimize(negative_ucb, x0=np.random.uniform(0, 1, dims), 
                 args=(gp, 1.5), bounds=bounds, method='L-BFGS-B')
    if result.success:
        results.append(result)
    
# Select best solution from restarts
next_point = min(results, key=lambda r: r.fun).x

print(f"Current best: {np.max(f2_outputs):.4f}")
print(f"Inputs producing current best: {f2_inputs[np.argmax(f2_outputs)]}")

# Select best solution from restarts
print("Next sample point (local optimizer):", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 5: 0.696738-0.300878

### Week 6

In [ ]:
#Load points to inputs and outputs from previous week
function = 2
week = 5 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f2_inputs = np.vstack([f2_inputs, row['provided_input'].values[0]])
f2_outputs = np.append(f2_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f2_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f2_inputs.shape}")
print(f"Inputs range: [{np.min(f2_inputs):.4f}, {np.max(f2_inputs):.4f}]")
print(f"Performance range: [{np.min(f2_outputs):.4e}, {np.max(f2_outputs):.4e}]")

This is best than last week, and close to the currently known best, but not better than previous outputs.

In [ ]:
x1 = f2_inputs[:, 0]
x2 = f2_inputs[:, 1]
y = f2_outputs

# Put data into a DataFrame
df = pd.DataFrame({
    "x1": x1,
    "x2": x2,
    "y": y
})

# Scatter plot with color encoding the output
plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x="x1", y="x2", hue="y", palette="coolwarm", s=60, legend=False)
plt.title("f2: Output as function of two inputs")
plt.xlabel("Input 1")
plt.ylabel("Input 2")
plt.xlim(0, 1)
plt.ylim(0, 1)
norm = plt.Normalize(vmin=y.min(), vmax=y.max())
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
plt.colorbar(sm, ax=plt.gca())
plt.show()


print(f"Evaluations: {len(f2_inputs)}")
print(f"Inputs shape: {f2_inputs.shape}")
print(f"Inputs range: [{np.min(f2_inputs):.4f}, {np.max(f2_inputs):.4f}]")
print(f"Performance range: [{np.min(f2_outputs):.4f}, {np.max(f2_outputs):.4f}]")
print(f"Current best: {np.max(f2_outputs):.4f}")
print(f"Inputs producing current best: {f2_inputs[np.argmax(f2_outputs)]}")

There is clearly something along the vertial line at x1 = 0.7 that indicates maxima along that axis. The max values are in the 0.6-0.7 range.

Similar to what I have tried with function 1, I will just sample one random point manually. This is how I have decided to spend this week's "bullet".

In [ ]:
chosen_point = np.array([0.9, 0.5])
print("-".join(f"{x:.6f}" for x in chosen_point.flatten()))

Sample for week 6: 0.900000-0.500000

## Function 3

This function is a function with a 1D output and a 3D input. 

This is the description of the function: you’re working on a drug discovery project, testing combinations of three compounds to create a new medicine.

Each experiment is stored in initial_inputs.npy as a 3D array, where each row lists the amounts of the three compounds used. After each experiment, you record the number of adverse reactions, stored in initial_outputs.npy as a 1D array.

Your goal is to minimise side effects; in this competition, it is framed as maximisation by optimising a transformed output (e.g. the negative of side effects). 

### Week 1

Let's observe the data.

In [ ]:
#print(f3_inputs)
#print(f3_outputs)

print(f"Evaluations: {len(f3_inputs)}")
print(f"Inputs shape: {f3_inputs.shape}")
print(f"Inputs range: [{np.min(f3_inputs):.4f}, {np.max(f3_inputs):.4f}]")
print(f"Performance range: [{np.min(f3_outputs):.4f}, {np.max(f3_outputs):.4f}]")
print(f"Current best: {np.max(f3_outputs):.4f}")
print(f"Inputs producing current best: {f3_inputs[np.argmax(f3_outputs)]}")


And even attempt to plot it.

In [ ]:
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(111, projection='3d')

sc = ax.scatter(f3_inputs[:,0], f3_inputs[:,1], f3_inputs[:,2], 
                c=f3_outputs, cmap='coolwarm', s=60)

ax.set_xlabel("Compound 1")
ax.set_ylabel("Compound 2")
ax.set_zlabel("Compound 3")
ax.set_title("Initial Data: Inputs vs Output")
cbar = fig.colorbar(sc, ax=ax, shrink=0.6)
cbar.set_label("Output (Side effects transformed)")
plt.show()

In [ ]:
# Let's follow a similar approach as with the others with a Matern and WhiteKerel

kernel = Matern(length_scale=[0.1, 0.1, 0.1], length_scale_bounds=(0.01, 1.0), nu=1.5) \
         + WhiteKernel(noise_level=1e-6)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True, n_restarts_optimizer=10)
gp.fit(f3_inputs, f3_outputs)

Getting these warnings:
* ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 1.0. Increasing the bound and calling fit again may find a better value.
* ConvergenceWarning: The optimal value found for dimension 1 of parameter k1__length_scale is close to the specified upper bound 1.0. Increasing the bound and calling fit again may find a better value.
* ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

In [ ]:
# Trying new hyperparameters, with wider length scale bounds, and introducing noise_level_bounds. amd more restarts optimizers, a constant kernel and wider bounds.

# Let's follow a similar approach as with the others with a Matern and WhiteKerel and a Constant Kernel (because I was getting many "close to bounds" messages)

kernel = C(1.0, (1e-5, 1e5)) * Matern(length_scale=[1.0, 1.0, 1.0], length_scale_bounds=(1e-5, 1e8), nu=2.5) \
         + WhiteKernel(noise_level=1e-8, noise_level_bounds=(1e-10, 1e0))
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True, n_restarts_optimizer=20, random_state=42)
gp.fit(f3_inputs, f3_outputs)

# And calculate over the whole cube

x1 = np.linspace(0, 1, 30)
x2 = np.linspace(0, 1, 30)
x3 = np.linspace(0, 1, 30)
X1, X2, X3 = np.meshgrid(x1, x2, x3)
X_candidate = np.column_stack([X1.ravel(), X2.ravel(), X3.ravel()])
y_mean, y_std = gp.predict(X_candidate, return_std=True)

# We will use Probability of improvement (PI) as an acquisition function with a penalty for being close to the boundary (was getting 1 1 0)
y_max = np.max(f3_outputs)
eta = 0.01
z = (y_mean - y_max - eta) / (y_std + 1e-12)
pi = stats.norm.cdf(z)
boundary_distances = np.minimum(X_candidate, 1 - X_candidate)
min_boundary_distance = np.min(boundary_distances, axis=1)
boundary_penalty = np.clip(min_boundary_distance / 0.1, 0, 1)
        
acquisition_function = pi * boundary_penalty

best_idx = np.argmax(acquisition_function)
next_point = X_candidate[best_idx]

print(f"Current best: {np.max(f3_outputs):.4f}")
print(f"Inputs producing current best: {f3_inputs[np.argmax(f3_outputs)]}")
print("Next point to sample based on PI:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 1: 0.172414-0.206897-0.206897

### Week 2

In [ ]:
#Load points to inputs and outputs from previous week
function = 3
week = 1 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f3_inputs = np.vstack([f3_inputs, row['provided_input'].values[0]])
f3_outputs = np.append(f3_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f3_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")

In [ ]:
print(f"Correlation between inputs: {np.corrcoef(f3_inputs, rowvar=False)}")

print('Means:', f3_inputs.mean(axis=0))
print('Stds:', f3_inputs.std(axis=0))
print('Mins:', f3_inputs.min(axis=0))
print('Maxs:', f3_inputs.max(axis=0))
print('Std/Mean Ratios:', f3_inputs.std(axis=0) / (f3_inputs.mean(axis=0) + 1e-12))

Let us try the same strategy again but with a UCB function

In [ ]:
kernel = C(1.0, (1e-5, 1e5)) * Matern(length_scale=[1.0, 1.0, 1.0], length_scale_bounds=(1e-5, 1e8), nu=2.5) \
         + WhiteKernel(noise_level=1e-8, noise_level_bounds=(1e-10, 1e0))
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True, n_restarts_optimizer=20, random_state=42)
gp.fit(f3_inputs, f3_outputs)


print(gp.kernel_)
#print(f3_inputs)

# And calculate over the whole cube

x1 = np.linspace(0, 1, 30)
x2 = np.linspace(0, 1, 30)
x3 = np.linspace(0, 1, 30)
X1, X2, X3 = np.meshgrid(x1, x2, x3)
X_candidate = np.column_stack([X1.ravel(), X2.ravel(), X3.ravel()])
y_mean, y_std = gp.predict(X_candidate, return_std=True)

# We will use Probability of improvement (PI) as an acquisition function with a penalty for being close to the boundary (was getting 1 1 0)
y_max = np.max(f3_outputs)
eta = 0.01
z = (y_mean - y_max - eta) / (y_std + 1e-12)
pi = stats.norm.cdf(z)
boundary_distances = np.minimum(X_candidate, 1 - X_candidate)
min_boundary_distance = np.min(boundary_distances, axis=1)
boundary_penalty = np.clip(min_boundary_distance / 0.1, 0, 1)
        
acquisition_function = pi * boundary_penalty

best_idx = np.argmax(acquisition_function)
next_point_pi = X_candidate[best_idx]

print(f"Current best: {np.max(f3_outputs):.4f}")
print(f"Inputs producing current best: {f3_inputs[np.argmax(f3_outputs)]}")
print("Next point to sample based on PI:", next_point_pi)

kappa = 0.5
UCB = y_mean + kappa * y_std  # shape = (grid_size, grid_size)
max_idx = np.argmax(UCB)  # index of the maximum UCB
next_point_ucb = X_candidate[max_idx]  # coordinates in input space
print("Next point to sample based on UCB:", next_point_ucb)


print("-".join(f"{x:.6f}" for x in next_point_ucb.flatten()))

Submission for week 2: 0.862069-0.137931-0.655172

### Week 3

In [ ]:
#Load points to inputs and outputs from previous week
function = 3
week = 2 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f3_inputs = np.vstack([f3_inputs, row['provided_input'].values[0]])
f3_outputs = np.append(f3_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f3_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f3_inputs.shape}")
print(f"Inputs range: [{np.min(f3_inputs):.4f}, {np.max(f3_inputs):.4f}]")
print(f"Performance range: [{np.min(f3_outputs):.4f}, {np.max(f3_outputs):.4f}]")

For this function I have been unable to make any improvement yet. This week I will try the same approach as for functions 1 and 2 with an EI acquisition function.

In [ ]:
# Define kernel: Matern with nu=2.5 for sharp peaks
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[0.2]*3, nu=2.5)

# Instantiate GP with small noise level
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True)

# Fit the GP model
gp.fit(f3_inputs, f3_outputs)
print(gp.kernel_)


# Function to optimize acquisition: Expected Improvement
def acquisitionf(x):
    x = np.array(x).reshape(1, -1)
    mean, std = gp.predict(x, return_std=True)
    # Current best observation
    best_y = np.max(f3_outputs)
    # Calculate EI
    z = (mean - best_y) / (std + 1e-9)
    ei = (mean - best_y) * norm.cdf(z) + std * norm.pdf(z)
    return -ei.item()  

# Robust acquisition maximization via multi-start local optimization
bounds = [(0, 1), (0, 1), (0, 1)]
num_restarts = 1500 # I have incremented this until the estimation did no longer move (fixed random seed)
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, 3)
    res = minimize(acquisitionf, x0, bounds=bounds, method='L-BFGS-B')
    results.append(res)

print(f"Current best: {np.max(f3_outputs):.4f}")
print(f"Inputs producing current best: {f3_inputs[np.argmax(f3_outputs)]}")

# Select best solution from restarts
next_point = min(results, key=lambda r: r.fun).x
print("Next sample point (local optimizer):", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 3: 0.507468-0.006386-0.305268

### Week 4

In [ ]:
#Load points to inputs and outputs from previous week
function = 3
week = 3 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f3_inputs = np.vstack([f3_inputs, row['provided_input'].values[0]])
f3_outputs = np.append(f3_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f3_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f3_inputs.shape}")
print(f"Inputs range: [{np.min(f3_inputs):.4f}, {np.max(f3_inputs):.4f}]")
print(f"Performance range: [{np.min(f3_outputs):.4f}, {np.max(f3_outputs):.4f}]")

I have yet to be able to further optimize this function beyond the known best values.

I will try an RBF kernel despite the description of sharp peaks.

In [ ]:
dims = 3

kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=[0.2]*dims, length_scale_bounds=(1e-10,1))

# Instantiate GP with small noise level
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True)

# Fit the GP model
gp.fit(f3_inputs, f3_outputs)
print(gp.kernel_)


# Function to optimize acquisition: Expected Improvement
def acquisitionf(x):
    x = np.array(x).reshape(1, -1)
    mean, std = gp.predict(x, return_std=True)
    # Current best observation
    best_y = np.max(f3_outputs)
    # Calculate EI
    z = (mean - best_y) / (std + 1e-9)
    ei = (mean - best_y) * norm.cdf(z) + std * norm.pdf(z)
    return -ei.item()  

# Robust acquisition maximization via multi-start local optimization
bounds = [(0, 1) for _ in range(dims)]
num_restarts = 150 
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, dims)
    res = minimize(acquisitionf, x0, bounds=bounds, method='L-BFGS-B')
    results.append(res)

print(f"Current best: {np.max(f3_outputs):.4e}")
print(f"Inputs producing current best: {f3_inputs[np.argmax(f3_outputs)]}")

# Select best solution from restarts
next_point_ei = min(results, key=lambda r: r.fun).x
print("Next sample point (ei):", next_point_ei)


def negative_ucb(x, gp, kappa):
    mu, sigma = gp.predict(x.reshape(1, -1), return_std=True)
    return -(mu[0] + kappa * sigma[0])

bounds = [(0, 1) for _ in range(dims)]
num_restarts = 150
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, dims)
    result = minimize(negative_ucb, x0=np.random.uniform(0, 1, dims), 
                 args=(gp, 1.7), bounds=bounds, method='L-BFGS-B')
    if result.success:
        results.append(result)
    
# Select best solution from restarts
next_point_ucb = min(results, key=lambda r: r.fun).x
print("Next sample point (ucb):", next_point_ucb)


print("-".join(f"{x:.6f}" for x in next_point_ei.flatten()))

I choose the EI value.

Submission for week 4: 0.589937-0.603952-0.435945

### Week 5

In [ ]:
#Load points to inputs and outputs from previous week
function = 3
week = 4 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f3_inputs = np.vstack([f3_inputs, row['provided_input'].values[0]])
f3_outputs = np.append(f3_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f3_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f3_inputs.shape}")
print(f"Inputs range: [{np.min(f3_inputs):.4f}, {np.max(f3_inputs):.4f}]")
print(f"Performance range: [{np.min(f3_outputs):.4f}, {np.max(f3_outputs):.4f}]")

This is a near 0 value! Remember that 0 is the goal. This week's submission should be focused on exploitation of this area. Last week we chose the highest EI, let's replicate the same again.

In [ ]:
dims = 3

#kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=[0.2]*dims, length_scale_bounds=(1e-10,10))
#kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[0.2]*dims, nu=1.5) + C(1.0, (1e-3, 1e3)) * RBF(length_scale=[0.2]*dims, length_scale_bounds=(1e-10,1))

kernel = C(1.0, (1e-5, 1e5)) * Matern(length_scale=[1.0, 1.0, 1.0], length_scale_bounds=(1e-5, 1e8), nu=2.5) \
         + WhiteKernel(noise_level=1e-8, noise_level_bounds=(1e-10, 1e0))
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True, n_restarts_optimizer=20, random_state=42)
# Instantiate GP with small noise level
#gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True)

# Fit the GP model
gp.fit(f3_inputs, f3_outputs)
print(gp.kernel_)


# Function to optimize acquisition: Expected Improvement
def acquisitionf(x):
    x = np.array(x).reshape(1, -1)
    mean, std = gp.predict(x, return_std=True)
    # Current best observation
    best_y = np.max(f3_outputs)
    # Calculate EI
    z = (mean - best_y) / (std + 1e-12)
    ei = (mean - best_y) * norm.cdf(z) + std * norm.pdf(z)
    return -ei.item()  

# Robust acquisition maximization via multi-start local optimization
bounds = [(0, 1) for _ in range(dims)]
num_restarts = 250 
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, dims)
    res = minimize(acquisitionf, x0, bounds=bounds, method='L-BFGS-B')
    results.append(res)

print(f"Current best: {np.max(f3_outputs):.4e}")
print(f"Inputs producing current best: {f3_inputs[np.argmax(f3_outputs)]}")

# Select best solution from restarts
next_point_ei = min(results, key=lambda r: r.fun).x
print("Next sample point (ei):", next_point_ei)


def negative_ucb(x, gp, kappa):
    mu, sigma = gp.predict(x.reshape(1, -1), return_std=True)
    return -(mu[0] + kappa * sigma[0])

bounds = [(0, 1) for _ in range(dims)]
num_restarts = 250
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, dims)
    result = minimize(negative_ucb, x0=np.random.uniform(0, 1, dims), 
                 args=(gp, 0.4), bounds=bounds, method='L-BFGS-B')
    if result.success:
        results.append(result)
    
# Select best solution from restarts
next_point_ucb = min(results, key=lambda r: r.fun).x
print("Next sample point (ucb):", next_point_ucb)


x1 = np.linspace(0, 1, 50)
x2 = np.linspace(0, 1, 50)
x3 = np.linspace(0, 1, 50)
X1, X2, X3 = np.meshgrid(x1, x2, x3)
X_candidate = np.column_stack([X1.ravel(), X2.ravel(), X3.ravel()])
y_mean, y_std = gp.predict(X_candidate, return_std=True)

# We will use Probability of improvement (PI) as an acquisition function with a penalty for being close to the boundary (was getting 1 1 0)
y_max = np.max(f3_outputs)
eta = 0.01
z = (y_mean - y_max - eta) / (y_std + 1e-12)
pi = stats.norm.cdf(z)
boundary_distances = np.minimum(X_candidate, 1 - X_candidate)
min_boundary_distance = np.min(boundary_distances, axis=1)
boundary_penalty = np.clip(min_boundary_distance / 0.1, 0, 1)
        
acquisition_function = pi * boundary_penalty

best_idx = np.argmax(acquisition_function)
next_point_pi = X_candidate[best_idx]

print("Next sample point (pi):", next_point_pi)


print(f"Current best: {np.max(f3_outputs):.4f}")
print(f"Inputs producing current best: {f3_inputs[np.argmax(f3_outputs)]}")
print("-".join(f"{x:.6f}" for x in next_point_ei.flatten()))

Submission for week 5: 0.593114-0.448446-0.293211

### Week 6

In [ ]:
#Load points to inputs and outputs from previous week
function = 3
week = 5 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f3_inputs = np.vstack([f3_inputs, row['provided_input'].values[0]])
f3_outputs = np.append(f3_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f3_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f3_inputs.shape}")
print(f"Inputs range: [{np.min(f3_inputs):.4f}, {np.max(f3_inputs):.4f}]")
print(f"Performance range: [{np.min(f3_outputs):.4f}, {np.max(f3_outputs):.4f}]")

I will try again the same strategy as we are very close to 0. EI as aquisition function.

In [ ]:
dims = 3

#kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=[0.2]*dims, length_scale_bounds=(1e-10,10))
#kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[0.2]*dims, nu=1.5) + C(1.0, (1e-3, 1e3)) * RBF(length_scale=[0.2]*dims, length_scale_bounds=(1e-10,1))

kernel = C(1.0, (1e-5, 1e5)) * Matern(length_scale=[1.0, 1.0, 1.0], length_scale_bounds=(1e-5, 1e8), nu=2.5) \
         + WhiteKernel(noise_level=1e-8, noise_level_bounds=(1e-10, 1e0))
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True, n_restarts_optimizer=20, random_state=42)
# Instantiate GP with small noise level
#gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True)

# Fit the GP model
gp.fit(f3_inputs, f3_outputs)
print(gp.kernel_)


# Function to optimize acquisition: Expected Improvement
def acquisitionf(x):
    x = np.array(x).reshape(1, -1)
    mean, std = gp.predict(x, return_std=True)
    # Current best observation
    best_y = np.max(f3_outputs)
    # Calculate EI
    z = (mean - best_y) / (std + 1e-12)
    ei = (mean - best_y) * norm.cdf(z) + std * norm.pdf(z)
    return -ei.item()  

# Robust acquisition maximization via multi-start local optimization
bounds = [(0, 1) for _ in range(dims)]
num_restarts = 250 
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, dims)
    res = minimize(acquisitionf, x0, bounds=bounds, method='L-BFGS-B')
    results.append(res)

print(f"Current best: {np.max(f3_outputs):.4e}")
print(f"Inputs producing current best: {f3_inputs[np.argmax(f3_outputs)]}")

# Select best solution from restarts
next_point_ei = min(results, key=lambda r: r.fun).x
print("Next sample point (ei):", next_point_ei)


def negative_ucb(x, gp, kappa):
    mu, sigma = gp.predict(x.reshape(1, -1), return_std=True)
    return -(mu[0] + kappa * sigma[0])

bounds = [(0, 1) for _ in range(dims)]
num_restarts = 250
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, dims)
    result = minimize(negative_ucb, x0=np.random.uniform(0, 1, dims), 
                 args=(gp, 0.4), bounds=bounds, method='L-BFGS-B')
    if result.success:
        results.append(result)
    
# Select best solution from restarts
next_point_ucb = min(results, key=lambda r: r.fun).x
print("Next sample point (ucb):", next_point_ucb)


x1 = np.linspace(0, 1, 50)
x2 = np.linspace(0, 1, 50)
x3 = np.linspace(0, 1, 50)
X1, X2, X3 = np.meshgrid(x1, x2, x3)
X_candidate = np.column_stack([X1.ravel(), X2.ravel(), X3.ravel()])
y_mean, y_std = gp.predict(X_candidate, return_std=True)

# We will use Probability of improvement (PI) as an acquisition function with a penalty for being close to the boundary (was getting 1 1 0)
y_max = np.max(f3_outputs)
eta = 0.01
z = (y_mean - y_max - eta) / (y_std + 1e-12)
pi = stats.norm.cdf(z)
boundary_distances = np.minimum(X_candidate, 1 - X_candidate)
min_boundary_distance = np.min(boundary_distances, axis=1)
boundary_penalty = np.clip(min_boundary_distance / 0.1, 0, 1)
        
acquisition_function = pi * boundary_penalty

best_idx = np.argmax(acquisition_function)
next_point_pi = X_candidate[best_idx]

print("Next sample point (pi):", next_point_pi)


print(f"Current best: {np.max(f3_outputs):.4f}")
print(f"Inputs producing current best: {f3_inputs[np.argmax(f3_outputs)]}")
print("-".join(f"{x:.6f}" for x in next_point_ei.flatten()))

Submission for week 6: 0.587870-0.097176-0.615007

## Function 4

This function is a function with a 1D output and a 4D input. 

This is the description of the function: Address the challenge of optimally placing products across warehouses for a business with high online sales, where accurate calculations are costly and only feasible biweekly. To speed up decision-making, an ML model approximates these results within hours. The model has four hyperparameters to tune, and its output reflects the difference from the expensive baseline. Because the system is dynamic and full of local optima, it requires careful tuning and robust validation to find reliable, near-optimal solutions. 

### Week 1

It is no longer viable to plot anything, so let's just look at the data:

In [ ]:
print(f"Evaluations: {len(f4_inputs)}")
print(f"Inputs shape: {f4_inputs.shape}")
print(f"Inputs range: [{np.min(f4_inputs):.4f}, {np.max(f4_inputs):.4f}]")
print(f"Performance range: [{np.min(f4_outputs):.4f}, {np.max(f4_outputs):.4f}]")
print(f"Current best: {np.max(f4_outputs):.4f}")
print(f"Inputs producing current best: {f4_inputs[np.argmax(f4_outputs)]}")


In [ ]:
kernel = (C(1.0, (1e-3, 1e3)) * Matern(length_scale=[2.0]*4, length_scale_bounds=(0.1, 1000.0), nu=2.5) + WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-15, 1e-1)))
gp = GaussianProcessRegressor(
        kernel=kernel,
        alpha=1e-6,
        normalize_y=True,
        n_restarts_optimizer=30,
        random_state=42
    )
gp.fit(f4_inputs, f4_outputs)

x1 = np.linspace(0, 1, 30)
x2 = np.linspace(0, 1, 30)
x3 = np.linspace(0, 1, 30)
x4 = np.linspace(0, 1, 30)
X1, X2, X3, X4 = np.meshgrid(x1, x2, x3, x4)
X_candidate = np.column_stack([X1.ravel(), X2.ravel(), X3.ravel(), X4.ravel()])

# Expected improvement
mu, sigma = gp.predict(X_candidate, return_std=True)
best = np.max(f4_outputs)
xi = 0.01
z = (mu - best - xi) / (sigma + 1e-12)
ei = (mu - best - xi) * stats.norm.cdf(z) + sigma * stats.norm.pdf(z)
ei

best_idx = np.argmax(ei)
next_point = X_candidate[best_idx]

print(f"Current best: {np.max(f4_outputs):.4f}")
print(f"Inputs producing current best: {f4_inputs[np.argmax(f4_outputs)]}")
print("Next point to sample based on PI:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 1: 0.448276-0.413793-0.379310-0.379310

### Week 2

In [ ]:
#Load points to inputs and outputs from previous week
function = 4
week = 1 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f4_inputs = np.vstack([f4_inputs, row['provided_input'].values[0]])
f4_outputs = np.append(f4_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f4_outputs):.4f}")
print(f"Previous best: {results_df[(results_df['week'] == week-1) & (results_df['function'] == function)]['known_best'].values[0]:.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")

This yielded a very good improvement. Let's replicate the method again this week, including the EI function.

In [ ]:
kernel = (C(1.0, (1e-3, 1e3)) * Matern(length_scale=[2.0]*4, length_scale_bounds=(0.1, 1000.0), nu=2.5) + WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-15, 1e-1)))
gp = GaussianProcessRegressor(
        kernel=kernel,
        alpha=1e-6,
        normalize_y=True,
        n_restarts_optimizer=30,
        random_state=42
    )
gp.fit(f4_inputs, f4_outputs)
print(gp.kernel_)

x1 = np.linspace(0, 1, 30)
x2 = np.linspace(0, 1, 30)
x3 = np.linspace(0, 1, 30)
x4 = np.linspace(0, 1, 30)
X1, X2, X3, X4 = np.meshgrid(x1, x2, x3, x4)
X_candidate = np.column_stack([X1.ravel(), X2.ravel(), X3.ravel(), X4.ravel()])

# Expected improvement
mu, sigma = gp.predict(X_candidate, return_std=True)
best = np.max(f4_outputs)
xi = 0.01
z = (mu - best - xi) / (sigma + 1e-12)
ei = (mu - best - xi) * stats.norm.cdf(z) + sigma * stats.norm.pdf(z)

best_idx = np.argmax(ei)
next_point = X_candidate[best_idx]

print(f"Current best: {np.max(f4_outputs):.4f}")
print(f"Inputs producing current best: {f4_inputs[np.argmax(f4_outputs)]}")
print("Next point to sample based on PI:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

The kernel is very balanced in terms of length scale for each dimension.

Submission for week 2: 0.413793-0.413793-0.344828-0.413793

### Week 3

In [ ]:
#Load points to inputs and outputs from previous week
function = 4
week = 2 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f4_inputs = np.vstack([f4_inputs, row['provided_input'].values[0]])
f4_outputs = np.append(f4_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f4_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f4_inputs.shape}")
print(f"Inputs range: [{np.min(f4_inputs):.4f}, {np.max(f4_inputs):.4f}]")
print(f"Performance range: [{np.min(f4_outputs):.4f}, {np.max(f4_outputs):.4f}]")

While this is the second improvement in a row, this function 4 has been sampled around the same point for two weeks in a row already.  This is encouraging exploitation. This week, for a change, I am using a UCB kernel encouraging exploration to see if I can find another local minimum.

In [ ]:
kernel = (C(1.0, (1e-3, 1e3)) * Matern(length_scale=[2.0]*4, length_scale_bounds=(0.1, 1000.0), nu=2.5) + WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-15, 1e-1)))
gp = GaussianProcessRegressor(
        kernel=kernel,
        alpha=1e-6,
        normalize_y=True,
        n_restarts_optimizer=30,
        random_state=42
    )
gp.fit(f4_inputs, f4_outputs)
print(gp.kernel_)

def negative_ucb(x, gp, kappa):
    mu, sigma = gp.predict(x.reshape(1, -1), return_std=True)
    return -(mu[0] + kappa * sigma[0])

bounds = [(0, 1) for _ in range(4)]
num_restarts = 50 
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, 4)
    #choosing kappa 3.7 to encourage exploration
    result = minimize(negative_ucb, x0=np.random.uniform(0, 1, 4), 
                 args=(gp, 3.7), bounds=bounds, method='L-BFGS-B')
    if result.success:
        results.append(result)
    
# Select best solution from restarts
next_point = min(results, key=lambda r: r.fun).x

print(f"Current best: {np.max(f4_outputs):.4f}")
print(f"Inputs producing current best: {f4_inputs[np.argmax(f4_outputs)]}")
print("Next sample point (local optimizer):", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))


Submission for week 3: 0.471861-0.446201-0.136768-0.434511

This is still exploiting this area. Next week I will just sample the point furthest from all known samples.

### Week 4

In [ ]:
#Load points to inputs and outputs from previous week
function = 4
week = 3 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f4_inputs = np.vstack([f4_inputs, row['provided_input'].values[0]])
f4_outputs = np.append(f4_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f4_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f4_inputs.shape}")
print(f"Inputs range: [{np.min(f4_inputs):.4f}, {np.max(f4_inputs):.4f}]")
print(f"Performance range: [{np.min(f4_outputs):.4f}, {np.max(f4_outputs):.4f}]")

This week the goal is just to sample the point that is further away from all known points to try and find another local minimum as the function descriptions describes it as having many and my strategy so far has been on exploitation.

In [ ]:
# Function to compute the minimum distance from a given point to the set of known points
def min_distance_to_points(point, points):
    # Compute pairwise distances between the point and each of the known points
    distances = cdist([point], points)
    # Return the minimum distance (further away from the closest point)
    return np.min(distances)

# Objective function to maximize the minimum distance
def objective_function(point, points):
    # We want to maximize the minimum distance, so we return the negative of the minimum distance
    return -min_distance_to_points(point, points)

# Bounds for the variables (each dimension is within [0,1])
bounds = [(0, 1)] * 4
num_restarts = 150
results = []
# Initial guess (can be any point in the space, e.g., the center of the space)
np.random.seed(42)
initial_guess = np.random.uniform(0, 1, 4)

for _ in range(num_restarts):
    res = minimize(objective_function, initial_guess, args=(f4_inputs,), bounds=bounds, method='L-BFGS-B')
    results.append(res)

# The resulting point is the one furthest from all known points
next_point = min(results, key=lambda r: r.fun).x

print(f"Current best: {np.max(f4_outputs):.4f}")
print(f"Inputs producing current best: {f4_inputs[np.argmax(f4_outputs)]}")
print("Next sample point (furthest point from known data):", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

This is clearly one corner that I am choosing to explore.

Submission for week 4: 0.571931-1.000000-1.000000-0.000000

### Week 5

In [ ]:
#Load points to inputs and outputs from previous week
function = 4
week = 4 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f4_inputs = np.vstack([f4_inputs, row['provided_input'].values[0]])
f4_outputs = np.append(f4_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f4_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f4_inputs.shape}")
print(f"Inputs range: [{np.min(f4_inputs):.4f}, {np.max(f4_inputs):.4f}]")
print(f"Performance range: [{np.min(f4_outputs):.4f}, {np.max(f4_outputs):.4f}]")

In [ ]:
f4_inputs_outputs = np.hstack([f4_inputs, f4_outputs.reshape(f4_inputs.shape[0],1)])
#print(f4_inputs_outputs)

print(f"Correlation between inputs and with outputs (last column): \n{np.corrcoef(f4_inputs_outputs, rowvar=False)}")


There is a negative correlation between all the inputs and the outputs.

Let's remember that this function is full of local optima. Because we are still in week 5, we can affort to explore a bit more.

I will try a dimensionality-reduction technique to plot and visualize what we`ve got so far.

In [ ]:
pca = PCA(n_components = 3)
f4_inputs_3d = pca.fit_transform(f4_inputs)

print(pca.explained_variance_ratio_)

So three dimentions explain about 0.35 + 0.30 + 0.22 = 0.87 of the variance. 

In [ ]:
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(111, projection='3d')

sc = ax.scatter(f4_inputs_3d[:,0], f4_inputs_3d[:,1], f4_inputs_3d[:,2], 
                c=f4_outputs, cmap='coolwarm', s=60)

ax.set_xlabel("z1")
ax.set_ylabel("z2")
ax.set_zlabel("z3")
ax.set_title("F4 3D dim reduction z components")
cbar = fig.colorbar(sc, ax=ax, shrink=0.6)
cbar.set_label("Output")
plt.show()

No local minima are observed but rather a cluster of values close to the maximum. 

I wanted to explore using a neural network this week, but given the results I think there is still some value in using a Gaussian process as a surrogate function once again considering the new extreme data point from last week.

Let's first explore where the furthest point lies now.

In [ ]:
num_restarts = 250

for _ in range(num_restarts):
    res = minimize(objective_function, initial_guess, args=(f4_inputs,), bounds=bounds, method='L-BFGS-B')
    results.append(res)

# The resulting point is the one furthest from all known points
next_point = min(results, key=lambda r: r.fun).x

print("Current furthest point from known data:", next_point)

No need to explore another corner. Let's use a GP with a Matern kernel and a softer nu. Still UCB, with a heavily exploratory kappa parameter.

In [ ]:
dims = 4

kernel = (C(1.0, (1e-3, 1e3)) * Matern(length_scale=[2.0]*dims, length_scale_bounds=(0.1, 1000.0), nu=1.5) + WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-15, 1e-1)))
gp = GaussianProcessRegressor(
        kernel=kernel,
        alpha=1e-6,
        normalize_y=True,
        n_restarts_optimizer=30,
        random_state=42
    )
gp.fit(f4_inputs, f4_outputs)
print(gp.kernel_)

def negative_ucb(x, gp, kappa):
    mu, sigma = gp.predict(x.reshape(1, -1), return_std=True)
    return -(mu[0] + kappa * sigma[0])

bounds = [(0, 1) for _ in range(dims)]
num_restarts = 50 
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, dims)
    result = minimize(negative_ucb, x0=np.random.uniform(0, 1, dims), 
                 args=(gp, 3.5), bounds=bounds, method='L-BFGS-B')
    if result.success:
        results.append(result)
    
# Select best solution from restarts
next_point = min(results, key=lambda r: r.fun).x

print(f"Current best: {np.max(f4_outputs):.4f}")
print(f"Inputs producing current best: {f4_inputs[np.argmax(f4_outputs)]}")
print("Next sample point (local optimizer):", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 5: 0.305616-0.434746-0.456803-0.473341

### Week 6

In [ ]:
#Load points to inputs and outputs from previous week
function = 4
week = 5 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f4_inputs = np.vstack([f4_inputs, row['provided_input'].values[0]])
f4_outputs = np.append(f4_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f4_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f4_inputs.shape}")
print(f"Inputs range: [{np.min(f4_inputs):.4f}, {np.max(f4_inputs):.4f}]")
print(f"Performance range: [{np.min(f4_outputs):.4f}, {np.max(f4_outputs):.4f}]")

## Function 5

This function is a function with a 1D output and a 4D input. 

This is the description of the function: You’re tasked with optimising a four-variable black-box function that represents the yield of a chemical process in a factory. The function is typically unimodal, with a single peak where yield is maximised. 

Your goal is to find the optimal combination of chemical inputs that delivers the highest possible yield, using systematic exploration and optimisation methods.

### Week 1

In [ ]:
print(f"Evaluations: {len(f5_inputs)}")
print(f"Inputs shape: {f5_inputs.shape}")
print(f"Inputs range: [{np.min(f5_inputs):.4f}, {np.max(f5_inputs):.4f}]")
print(f"Performance range: [{np.min(f5_outputs):.4f}, {np.max(f5_outputs):.4f}]")
print(f"Current best: {np.max(f5_outputs):.4f}")
print(f"Inputs producing current best: {f5_inputs[np.argmax(f5_outputs)]}")

In [ ]:
# Function is unimodal with a single peak, so I am choosing an RBF kernel
kernel = RBF(length_scale=[1.0] * 4, length_scale_bounds=(0.05, 15000.0)) 
gp = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=15,
    alpha=1e-6, #modelling noise     
    normalize_y=True,
    random_state=42
)
gp.fit(f5_inputs, f5_outputs)

x1 = np.linspace(0, 1, 30)
x2 = np.linspace(0, 1, 30)
x3 = np.linspace(0, 1, 30)
x4 = np.linspace(0, 1, 30)
X1, X2, X3, X4 = np.meshgrid(x1, x2, x3, x4)
X_candidate = np.column_stack([X1.ravel(), X2.ravel(), X3.ravel(), X4.ravel()])

# Expected improvement
y_mean, y_std = gp.predict(X_candidate, return_std=True)

# The fact that the function has a single peak also makes me avoid PI, and a simple UCB should be fine
kappa = 2.0
UCB = y_mean + kappa * y_std 

max_idx = np.argmax(UCB)  # index of the maximum UCB
next_point = X_candidate[max_idx]  # coordinates in input space

print(f"Current best: {np.max(f5_outputs):.4f}")
print(f"Inputs producing current best: {f5_inputs[np.argmax(f5_outputs)]}")
print("Next point to sample based on UCB:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 1: 0.586207-0.620690-1.000000-0.931034

It is noted that I am one value at one of the bounds, will explore next week.

### Week 2

In [ ]:
#Load points to inputs and outputs from previous week
function = 5
week = 1 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f5_inputs = np.vstack([f5_inputs, row['provided_input'].values[0]])
f5_outputs = np.append(f5_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f5_outputs):.4f}")
print(f"Previous best: {results_df[(results_df['week'] == week-1) & (results_df['function'] == function)]['known_best'].values[0]:.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")

This yielded a very good improvement, what this function maximum is is unknown.

Let's replicate again the same method this week

In [ ]:
# Function is unimodal with a single peak, so I am choosing an RBF kernel
kernel = RBF(length_scale=[1.0] * 4, length_scale_bounds=(0.05, 15000.0)) 
gp = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=15,
    alpha=1e-6, #modelling noise     
    normalize_y=True,
    random_state=42
)
gp.fit(f5_inputs, f5_outputs)

print(gp.kernel_)

x1 = np.linspace(0, 1, 30)
x2 = np.linspace(0, 1, 30)
x3 = np.linspace(0, 1, 30)
x4 = np.linspace(0, 1, 30)
X1, X2, X3, X4 = np.meshgrid(x1, x2, x3, x4)
X_candidate = np.column_stack([X1.ravel(), X2.ravel(), X3.ravel(), X4.ravel()])

# Expected improvement
y_mean, y_std = gp.predict(X_candidate, return_std=True)

# The fact that the function has a single peak also makes me avoid PI, and a simple UCB should be fine
kappa = 0.5
UCB = y_mean + kappa * y_std 

max_idx = np.argmax(UCB)  # index of the maximum UCB
next_point = X_candidate[max_idx]  # coordinates in input space

print(f"Current best: {np.max(f5_outputs):.4f}")
print(f"Inputs producing current best: {f5_inputs[np.argmax(f5_outputs)]}")
print("Next point to sample based on UCB:", next_point)

This would be encouraging sampling on a corner. While I may try this once, I will first try for this week's submission:
* Reducing the boundaries so that extreme values are not suggested.
* A denser mesh as 4 dimensions is still manageable.
* A penalty for being close to the borders (not linear but harsh around the border)

In [ ]:
# If you want to avoid sampling exactly at the edges
x1 = np.linspace(0.05, 0.95, 50)
x2 = np.linspace(0.05, 0.95, 50)
x3 = np.linspace(0.05, 0.95, 50)
x4 = np.linspace(0.05, 0.95, 50)
X1, X2, X3, X4 = np.meshgrid(x1, x2, x3, x4)
X_candidate = np.column_stack([X1.ravel(), X2.ravel(), X3.ravel(), X4.ravel()])

y_mean, y_std = gp.predict(X_candidate, return_std=True)

kappa = 0.5
UCB = y_mean + kappa * y_std
#edge_penalty = 0.5 * ((X_candidate < 0.07) | (X_candidate > 0.93)).sum(axis=1)
#UCB_adjusted = UCB - edge_penalty

# Let's add a penalty for getting close to the borders
# Calculate distance to nearest boundary for each dimension (0 and 1 here)
min_boundary_distance = np.minimum(np.min(X_candidate, axis=1), np.min(1 - X_candidate, axis=1))
min_boundary_distance.shape

# Penalty factor: linearly scale with distance from boundary (0 at boundary, 1 at distance >= penalty_radius)
penalty_radius = 0.1
penalty_factor = np.clip(min_boundary_distance / penalty_radius, 0, 1)

# Apply penalty
UCB_penalized = UCB * penalty_factor

max_idx = np.argmax(UCB_penalized)
next_point_2 = X_candidate[max_idx]

print(f"Current best: {np.max(f5_outputs):.4f}")
print(f"Inputs producing current best: {f5_inputs[np.argmax(f5_outputs)]}")
print("Next point to sample based on UCB:", next_point_2)

Considering this, I will choose to sample the extreme this week and, depending on the result, reevaluate next week.

In [ ]:
print("Next point to sample based on UCB:", next_point_2)
print("-".join(f"{x:.6f}" for x in next_point_2.flatten()))

Submission for week 2: 0.894898-0.105102-0.894898-0.894898

### Week 3

In [ ]:
#Load points to inputs and outputs from previous week
function = 5
week = 2 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f5_inputs = np.vstack([f5_inputs, row['provided_input'].values[0]])
f5_outputs = np.append(f5_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f5_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f5_inputs.shape}")
print(f"Inputs range: [{np.min(f5_inputs):.4f}, {np.max(f5_inputs):.4f}]")
print(f"Performance range: [{np.min(f5_outputs):.4f}, {np.max(f5_outputs):.4f}]")

In [ ]:
#Switching to Mattern kernel
 
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[0.2]*4, nu=2.5) \
         + WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-8, 1e0))
gp = GaussianProcessRegressor(
        kernel=kernel,
        alpha=1e-6,
        normalize_y=True,
        n_restarts_optimizer=30,
        random_state=42
    )
gp.fit(f4_inputs, f4_outputs)
print(gp.kernel_)

def negative_ucb(x, gp, kappa):
    mu, sigma = gp.predict(x.reshape(1, -1), return_std=True)
    return -(mu[0] + kappa * sigma[0])

bounds = [(0, 1) for _ in range(4)]
num_restarts = 50
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, 4)
    #choosing kappa 2.7 
    result = minimize(negative_ucb, x0=np.random.uniform(0, 1, 4), 
                 args=(gp, 2.7), bounds=bounds, method='L-BFGS-B')
    if result.success:
        results.append(result)
    
# Select best solution from restarts
next_point = min(results, key=lambda r: r.fun).x

print(f"Current best: {np.max(f5_outputs):.4f}")
print(f"Inputs producing current best: {f5_inputs[np.argmax(f5_outputs)]}")
print("Next sample point (local optimizer):", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 3: 0.452640-0.433354-0.202919-0.441704

### Week 4

In [ ]:
#Load points to inputs and outputs from previous week
function = 5
week = 3 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f5_inputs = np.vstack([f5_inputs, row['provided_input'].values[0]])
f5_outputs = np.append(f5_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f5_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f5_inputs.shape}")
print(f"Inputs range: [{np.min(f5_inputs):.4f}, {np.max(f5_inputs):.4f}]")
print(f"Performance range: [{np.min(f5_outputs):.4f}, {np.max(f5_outputs):.4f}]")

This is not an improvement, but this allows me to capture yet another datapoint that will be useful.

I am choosing to apply the same methodology once again this week, but reverting to an RBF kernel.

In [ ]:
#Switching to Mattern kernel

kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=[1.0] * 4, length_scale_bounds=(0.05, 15000.0)) 
#kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[0.2]*4, nu=2.5) \
#         + WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-8, 1e0))
gp = GaussianProcessRegressor(
        kernel=kernel,
        alpha=1e-6,
        normalize_y=True,
        n_restarts_optimizer=30,
        random_state=42
    )
gp.fit(f4_inputs, f4_outputs)
print(gp.kernel_)

def negative_ucb(x, gp, kappa):
    mu, sigma = gp.predict(x.reshape(1, -1), return_std=True)
    return -(mu[0] + kappa * sigma[0])

bounds = [(0, 1) for _ in range(4)]
num_restarts = 150
results = []
np.random.seed(42)

for _ in range(num_restarts):
    x0 = np.random.uniform(0, 1, 4)
    #choosing kappa 1.5
    result = minimize(negative_ucb, x0=np.random.uniform(0, 1, 4), 
                 args=(gp, 1.5), bounds=bounds, method='L-BFGS-B')
    if result.success:
        results.append(result)
    
# Select best solution from restarts
next_point = min(results, key=lambda r: r.fun).x

print(f"Current best: {np.max(f5_outputs):.4f}")
print(f"Inputs producing current best: {f5_inputs[np.argmax(f5_outputs)]}")
print("Next sample point (local optimizer):", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 4: 0.017701-0.472569-0.492069-0.597183

### Week 5

In [ ]:
#Load points to inputs and outputs from previous week
function = 5
week = 4 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f5_inputs = np.vstack([f5_inputs, row['provided_input'].values[0]])
f5_outputs = np.append(f5_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f5_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f5_inputs.shape}")
print(f"Inputs range: [{np.min(f5_inputs):.4f}, {np.max(f5_inputs):.4f}]")
print(f"Performance range: [{np.min(f5_outputs):.4f}, {np.max(f5_outputs):.4f}]")

In [ ]:
f5_inputs_outputs = np.hstack([f5_inputs, f5_outputs.reshape(f5_inputs.shape[0],1)])
#print(f5_inputs_outputs)

print(f"Correlation between inputs and with outputs (last column): \n{np.corrcoef(f5_inputs_outputs, rowvar=False)}")

Results were better with a RBF kernel. Function is unimodal and there is a single peak.

In [ ]:
# Function is unimodal with a single peak, so I am choosing an RBF kernel
kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=[1.0] * 4, length_scale_bounds=(0.05, 15000.0)) 
gp = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=25,
    alpha=1e-6, #modelling noise     
    normalize_y=True,
    random_state=42
)
gp.fit(f5_inputs, f5_outputs)

print(gp.kernel_)

x1 = np.linspace(0, 1, 30)
x2 = np.linspace(0, 1, 30)
x3 = np.linspace(0, 1, 30)
x4 = np.linspace(0, 1, 30)
X1, X2, X3, X4 = np.meshgrid(x1, x2, x3, x4)
X_candidate = np.column_stack([X1.ravel(), X2.ravel(), X3.ravel(), X4.ravel()])

# Expected improvement
y_mean, y_std = gp.predict(X_candidate, return_std=True)

# The fact that the function has a single peak also makes me avoid PI, and a simple UCB should be fine
kappa = 0.3
UCB = y_mean + kappa * y_std 

max_idx = np.argmax(UCB)  # index of the maximum UCB
next_point = X_candidate[max_idx]  # coordinates in input space

print(f"Current best: {np.max(f5_outputs):.4f}")
print(f"Inputs producing current best: {f5_inputs[np.argmax(f5_outputs)]}")
print("Next point to sample based on UCB:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 5: 1.000000-0.000000-1.000000-0.965517

### Week 6

In [ ]:
#Load points to inputs and outputs from previous week
function = 5
week = 5 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f5_inputs = np.vstack([f5_inputs, row['provided_input'].values[0]])
f5_outputs = np.append(f5_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f5_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f5_inputs.shape}")
print(f"Inputs range: [{np.min(f5_inputs):.4f}, {np.max(f5_inputs):.4f}]")
print(f"Performance range: [{np.min(f5_outputs):.4f}, {np.max(f5_outputs):.4f}]")

## Function 6

This function is a function with a 1D output and a 5D input. 

This is the description of the function: You’re optimising a cake recipe using a black-box function with five ingredient inputs, for example flour, sugar, eggs, butter and milk. Each recipe is evaluated with a combined score based on flavour, consistency, calories, waste and cost, where each factor contributes negative points as judged by an expert taster. This means the total score is negative by design. 

To frame this as a maximisation problem, your goal is to bring that score as close to zero as possible or, equivalently, to maximise the negative of the total sum.

### Week 1


In [ ]:
print(f"Evaluations: {len(f6_inputs)}")
print(f"Inputs shape: {f6_inputs.shape}")
print(f"Inputs range: [{np.min(f6_inputs):.4f}, {np.max(f6_inputs):.4f}]")
print(f"Performance range: [{np.min(f6_outputs):.4f}, {np.max(f6_outputs):.4f}]")
print(f"Current best: {np.max(f6_outputs):.4f}")
print(f"Inputs producing current best: {f6_inputs[np.argmax(f6_outputs)]}")


In [ ]:
kernel = RBF(length_scale=[1.0] * 5, length_scale_bounds=(0.05, 15000.0)) 
gp = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=15,
    alpha=1e-6, #modelling noise     
    normalize_y=True,
    random_state=42
)
gp.fit(f6_inputs, f6_outputs)

x1 = np.linspace(0, 1, 30)
x2 = np.linspace(0, 1, 30)
x3 = np.linspace(0, 1, 30)
x4 = np.linspace(0, 1, 30)
x5 = np.linspace(0, 1, 30)
X1, X2, X3, X4, X5 = np.meshgrid(x1, x2, x3, x4, x5)
X_candidate = np.column_stack([X1.ravel(), X2.ravel(), X3.ravel(), X4.ravel(), X5.ravel()])

# Expected improvement
y_mean, y_std = gp.predict(X_candidate, return_std=True)

# The fact that the function has a single peak also makes me avoid PI, and a simple UCB should be fine
kappa = 2.0
UCB = y_mean + kappa * y_std 

max_idx = np.argmax(UCB)  # index of the maximum UCB
next_point = X_candidate[max_idx]  # coordinates in input space

print(f"Current best: {np.max(f6_outputs):.4f}")
print(f"Inputs producing current best: {f6_inputs[np.argmax(f6_outputs)]}")
print("Next point to sample based on UCB:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 1 0.000000-0.655172-0.103448-0.931034-0.275862

It is noted that I am one value at one of the bounds, will explore next week.

### Week 2

In [ ]:
#Load points to inputs and outputs from previous week
function = 6
week = 1 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f6_inputs = np.vstack([f6_inputs, row['provided_input'].values[0]])
f6_outputs = np.append(f6_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f6_outputs):.4f}")
print(f"Previous best: {results_df[(results_df['week'] == week-1) & (results_df['function'] == function)]['known_best'].values[0]:.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")

The ideal score is zero, which we are still far from.

In [ ]:
print(f"Correlation between inputs: \n{np.corrcoef(f6_inputs, rowvar=False)}\n")

print('Means:', f6_inputs.mean(axis=0))
print('Stds:', f6_inputs.std(axis=0))
print('Mins:', f6_inputs.min(axis=0))
print('Maxs:', f6_inputs.max(axis=0))
print('Std/Mean Ratios:', f6_inputs.std(axis=0) / (f6_inputs.mean(axis=0) + 1e-12))

Let's try to get a better fit of a surrogate function and willingly limit the upper bound of length scale, and a smaller kappa for UCB.

In [ ]:
# Function is unimodal with a single peak, so I am choosing an RBF kernel
kernel = RBF(length_scale=[1.0] * 5, length_scale_bounds=(0.05, 15)) 
gp = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=15,
    alpha=1e-6, #modelling noise     
    normalize_y=True,
    random_state=42
)
gp.fit(f6_inputs, f6_outputs)
print(gp.kernel_)

x1 = np.linspace(0, 1, 30)
x2 = np.linspace(0, 1, 30)
x3 = np.linspace(0, 1, 30)
x4 = np.linspace(0, 1, 30)
x5 = np.linspace(0, 1, 30)
X1, X2, X3, X4, X5 = np.meshgrid(x1, x2, x3, x4, x5)
X_candidate = np.column_stack([X1.ravel(), X2.ravel(), X3.ravel(), X4.ravel(), X5.ravel()])

# Expected improvement
y_mean, y_std = gp.predict(X_candidate, return_std=True)

# The fact that the function has a single peak also makes me avoid PI, and a simple UCB should be fine
kappa = 0.5
UCB = y_mean + kappa * y_std 

max_idx = np.argmax(UCB)  # index of the maximum UCB
next_point = X_candidate[max_idx]  # coordinates in input space

print(f"Current best: {np.max(f6_outputs):.4f}")
print(f"Inputs producing current best: {f6_inputs[np.argmax(f6_outputs)]}")
print("Next point to sample based on UCB:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

#print(f6_inputs)

Submission for week 2: 0.517241-0.310345-0.517241-0.862069-0.103448

### Week 3

In [ ]:
#Load points to inputs and outputs from previous week
function = 6
week = 2 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f6_inputs = np.vstack([f6_inputs, row['provided_input'].values[0]])
f6_outputs = np.append(f6_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f6_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f6_inputs.shape}")
print(f"Inputs range: [{np.min(f6_inputs):.4f}, {np.max(f6_inputs):.4f}]")
print(f"Performance range: [{np.min(f6_outputs):.4f}, {np.max(f6_outputs):.4f}]")

In [ ]:
kernel = RBF(length_scale=[1.0] * 5, length_scale_bounds=(0.05, 15)) 
gp = GaussianProcessRegressor(
        kernel=kernel,
        alpha=1e-6,
        normalize_y=True,
        n_restarts_optimizer=30,
        random_state=42
    )
gp.fit(f6_inputs, f6_outputs)
print(gp.kernel_)

def negative_ucb(x, gp, kappa):
    mu, sigma = gp.predict(x.reshape(1, -1), return_std=True)
    return -(mu[0] + kappa * sigma[0])

def neg_ei(x):
    x = np.array(x).reshape(1, -1)
    mean, std = gp.predict(x, return_std=True)
    # Current best observation
    best_y = np.max(f6_outputs)
    # Calculate EI
    z = (mean - best_y) / (std + 1e-9)
    ei = (mean - best_y) * norm.cdf(z) + std * norm.pdf(z)
    return -ei.item()  


bounds = [(0, 1) for _ in range(5)]
num_restarts = 100
results = []
np.random.seed(42)

for _ in range(num_restarts):
    #choosing kappa 2.7 
    result = minimize(negative_ucb, x0=np.random.uniform(0, 1, 5), 
                 args=(gp, 2.7), bounds=bounds, method='L-BFGS-B')
    if result.success:
        results.append(result)
    
# Select best solution from restarts
next_point = min(results, key=lambda r: r.fun).x
print("Next sample point (local optimizer) based on UCB:", next_point)

num_restarts = 50
results = []
np.random.seed(42)

for _ in range(num_restarts):
    #choosing kappa 2.7 
    result = minimize(neg_ei, x0=np.random.uniform(0, 1, 5), 
                 bounds=bounds, method='L-BFGS-B')
    if result.success:
        results.append(result)

next_point = min(results, key=lambda r: r.fun).x

print(f"Current best: {np.max(f6_outputs):.4f}")
print(f"Inputs producing current best: {f6_inputs[np.argmax(f6_outputs)]}")
print("Next sample point (local optimizer) based on EI:", next_point)

print("-".join(f"{x:.6f}" for x in next_point.flatten()))

I choose the EI point.

Submission for week 3: 0.493775-0.266439-0.665329-1.000000-0.266425

### Week 4

In [ ]:
#Load points to inputs and outputs from previous week
function = 6
week = 3 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f6_inputs = np.vstack([f6_inputs, row['provided_input'].values[0]])
f6_outputs = np.append(f6_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f6_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f6_inputs.shape}")
print(f"Inputs range: [{np.min(f6_inputs):.4f}, {np.max(f6_inputs):.4f}]")
print(f"Performance range: [{np.min(f6_outputs):.4f}, {np.max(f6_outputs):.4f}]")

In [ ]:
kernel = RBF(length_scale=[1.0] * 5, length_scale_bounds=(0.05, 15)) 
gp = GaussianProcessRegressor(
        kernel=kernel,
        alpha=1e-6,
        normalize_y=True,
        n_restarts_optimizer=30,
        random_state=42
    )
gp.fit(f6_inputs, f6_outputs)
print(gp.kernel_)

def negative_ucb(x, gp, kappa):
    mu, sigma = gp.predict(x.reshape(1, -1), return_std=True)
    return -(mu[0] + kappa * sigma[0])

def neg_ei(x):
    x = np.array(x).reshape(1, -1)
    mean, std = gp.predict(x, return_std=True)
    # Current best observation
    best_y = np.max(f6_outputs)
    # Calculate EI
    z = (mean - best_y) / (std + 1e-9)
    ei = (mean - best_y) * norm.cdf(z) + std * norm.pdf(z)
    return -ei.item()  


bounds = [(0, 1) for _ in range(5)]
num_restarts = 100
results = []
np.random.seed(42)

for _ in range(num_restarts):
    #choosing kappa 2.7 
    result = minimize(negative_ucb, x0=np.random.uniform(0, 1, 5), 
                 args=(gp, 1.2), bounds=bounds, method='L-BFGS-B')
    if result.success:
        results.append(result)
    
# Select best solution from restarts
next_point_ucb = min(results, key=lambda r: r.fun).x
print("Next sample point (local optimizer) based on UCB:", next_point_ucb)

num_restarts = 200
results = []
np.random.seed(42)

for _ in range(num_restarts):
    result = minimize(neg_ei, x0=np.random.uniform(0, 1, 5), 
                 bounds=bounds, method='L-BFGS-B')
    if result.success:
        results.append(result)

next_point_ei = min(results, key=lambda r: r.fun).x

print(f"Current best: {np.max(f6_outputs):.4f}")
print(f"Inputs producing current best: {f6_inputs[np.argmax(f6_outputs)]}")
print("Next sample point (local optimizer) based on EI:", next_point_ei)

print("-".join(f"{x:.6f}" for x in next_point_ucb.flatten()))

I choose the UCB estimate.

Submission for week 4: 0.482310-0.418871-0.201310-0.981559-0.000000

### Week 5

In [ ]:
#Load points to inputs and outputs from previous week
function = 6
week = 4 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f6_inputs = np.vstack([f6_inputs, row['provided_input'].values[0]])
f6_outputs = np.append(f6_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f6_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f6_inputs.shape}")
print(f"Inputs range: [{np.min(f6_inputs):.4f}, {np.max(f6_inputs):.4f}]")
print(f"Performance range: [{np.min(f6_outputs):.4f}, {np.max(f6_outputs):.4f}]")

In [ ]:
f6_inputs_outputs = np.hstack([f6_inputs, f6_outputs.reshape(f6_inputs.shape[0],1)])
#print(f6_inputs_outputs)

print(f"Correlation between inputs and with outputs (last column): \n{np.corrcoef(f6_inputs_outputs, rowvar=False)}")

Let us replicate what was done on week 2.

In [ ]:
# Function is unimodal with a single peak, so I am choosing an RBF kernel
kernel = C(1.0, (1e-6, 1e6)) * RBF(length_scale=[1.0] * 5, length_scale_bounds=(0.05, 15)) 
gp = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=15,
    alpha=1e-6, #modelling noise     
    normalize_y=True,
    random_state=42
)
gp.fit(f6_inputs, f6_outputs)
print(gp.kernel_)

x1 = np.linspace(0, 1, 30)
x2 = np.linspace(0, 1, 30)
x3 = np.linspace(0, 1, 30)
x4 = np.linspace(0, 1, 30)
x5 = np.linspace(0, 1, 30)
X1, X2, X3, X4, X5 = np.meshgrid(x1, x2, x3, x4, x5)
X_candidate = np.column_stack([X1.ravel(), X2.ravel(), X3.ravel(), X4.ravel(), X5.ravel()])

# Expected improvement
y_mean, y_std = gp.predict(X_candidate, return_std=True)

# The fact that the function has a single peak also makes me avoid PI, and a simple UCB should be fine
kappa = 0.3
UCB = y_mean + kappa * y_std 

max_idx = np.argmax(UCB)  # index of the maximum UCB
next_point = X_candidate[max_idx]  # coordinates in input space

print(f"Current best: {np.max(f6_outputs):.4f}")
print(f"Inputs producing current best: {f6_inputs[np.argmax(f6_outputs)]}")
print("Next point to sample based on UCB:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

#print(f6_inputs)

Submission for week 5: 0.241379-0.310345-0.689655-0.931034-0.103448

### Week 6

In [ ]:
#Load points to inputs and outputs from previous week
function = 6
week = 5 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f6_inputs = np.vstack([f6_inputs, row['provided_input'].values[0]])
f6_outputs = np.append(f6_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f6_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f6_inputs.shape}")
print(f"Inputs range: [{np.min(f6_inputs):.4f}, {np.max(f6_inputs):.4f}]")
print(f"Performance range: [{np.min(f6_outputs):.4f}, {np.max(f6_outputs):.4f}]")

## Function 7

This function is a function with a 1D output and a 6D input. 

This is the description of the function: You’re tasked with optimising an ML model by tuning six hyperparameters, for example learning rate, regularisation strength or number of hidden layers. The function you’re maximising is the model’s performance score (such as accuracy or F1), but since the relationship between inputs and output isn’t known, it’s treated as a black-box function. 

Because this is a commonly used model, you might benefit from researching best practices or literature to guide your initial search space. Your goal is to find the combination of hyperparameters that yields the highest possible performance.

### Week 1

The problem statement is similar to function 2, but with more dimensions.

In [ ]:
print(f"Evaluations: {len(f7_inputs)}")
print(f"Inputs shape: {f7_inputs.shape}")
print(f"Inputs range: [{np.min(f7_inputs):.4f}, {np.max(f7_inputs):.4f}]")
print(f"Performance range: [{np.min(f7_outputs):.4f}, {np.max(f7_outputs):.4f}]")
print(f"Current best: {np.max(f7_outputs):.4f}")
print(f"Inputs producing current best: {f7_inputs[np.argmax(f7_outputs)]}")


There is not known relationship between the parameters and the output. I will try a combination of Mattern, Constant, RBF and White with UCB as adcquisition function.

In [ ]:
kernel = (
    C(1.0, (1e-6, 1e6)) * (
        RBF([1.0]*6, length_scale_bounds=(1e-2, 1e10)) +
        Matern(length_scale=[1.0]*6, length_scale_bounds=(1e-2,1e13), nu=2.5)
    ) +
    WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-13, 1e-1))
)

gp = GaussianProcessRegressor(kernel=kernel,
                              alpha=1e-6,
                              normalize_y=True,
                              n_restarts_optimizer=20,
                              random_state=42)
gp.fit(f7_inputs, f7_outputs)

margin = 0.01  # avoid boundaries, previous function gave a 1 for one of the coordinates
#grid_points = 18 # to be able to lower it as we're exploring grip_points^6, 20 was giving memory errors
#axes = [np.linspace(margin, 1-margin, grid_points) for _ in range(6)]
#mesh = np.meshgrid(*axes) # instead of doing this 6 times
#X_candidates = np.column_stack([arr.ravel() for arr in mesh])

#y_mean, y_std = gp.predict(X_candidates, return_std=True)

# Random sampling approach to avoid memory issues
n_candidates = 1000000  # Much more manageable
np.random.seed(42)
X_candidates = np.random.uniform(
    low=margin, 
    high=1-margin, 
    size=(n_candidates, 6)
)

y_mean, y_std = gp.predict(X_candidates, return_std=True)

# UCB acquisition function
kappa = 2.5 
UCB = y_mean + kappa * y_std 

next_idx = np.argmax(UCB)
next_point = X_candidates[next_idx]

print(f"Current best: {np.max(f7_outputs):.4f}")
print(f"Inputs producing current best: {f7_inputs[np.argmax(f7_outputs)]}")
print("Next point to sample based on UCB:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))


Submission for the week: 0.014182-0.276863-0.738085-0.053110-0.375860-0.799450

### Week 2

In [ ]:
#Load points to inputs and outputs from previous week
function = 7
week = 1 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f7_inputs = np.vstack([f7_inputs, row['provided_input'].values[0]])
f7_outputs = np.append(f7_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f7_outputs):.4f}")
print(f"Previous best: {results_df[(results_df['week'] == week-1) & (results_df['function'] == function)]['known_best'].values[0]:.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")

Marginal improvement as compared to last week.

In [ ]:
print(f"Correlation between inputs: \n{np.corrcoef(f7_inputs, rowvar=False)}\n")

print('Means:', f7_inputs.mean(axis=0))
print('Stds:', f7_inputs.std(axis=0))
print('Mins:', f7_inputs.min(axis=0))
print('Maxs:', f7_inputs.max(axis=0))
print('Std/Mean Ratios:', f7_inputs.std(axis=0) / (f7_inputs.mean(axis=0) + 1e-12))

In [ ]:
kernel = (
    C(1.0, (1e-2, 1e2)) * (
        RBF([1.0]*6, length_scale_bounds=(1e-2, 3)) +
        Matern(length_scale=[1.0]*6, length_scale_bounds=(1e-2,1e2), nu=2.5)
    ) +
    WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-13, 1e-1))
)

gp = GaussianProcessRegressor(kernel=kernel,
                              alpha=1e-6,
                              normalize_y=True,
                              n_restarts_optimizer=20,
                              random_state=42)
gp.fit(f7_inputs, f7_outputs)
print(gp.kernel_)

# Random sampling approach to avoid memory issues
n_candidates = 3000000  # Much more manageable
np.random.seed(42)
margin=0.01
X_candidates = np.random.uniform(
    low=margin, 
    high=1-margin, 
    size=(n_candidates, 6)
)

y_mean, y_std = gp.predict(X_candidates, return_std=True)

# UCB acquisition function
kappa = 0.5 
UCB = y_mean + kappa * y_std 

next_idx = np.argmax(UCB)
next_point = X_candidates[next_idx]

print(f"Current best: {np.max(f7_outputs):.4f}")
print(f"Inputs producing current best: {f7_inputs[np.argmax(f7_outputs)]}")
print("Next point to sample based on UCB:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))


Submission for week 2: 0.016974-0.324003-0.154628-0.172541-0.388943-0.758898

### Week 3

In [ ]:
#Load points to inputs and outputs from previous week
function = 7
week = 2 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f7_inputs = np.vstack([f7_inputs, row['provided_input'].values[0]])
f7_outputs = np.append(f7_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f7_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f7_inputs.shape}")
print(f"Inputs range: [{np.min(f7_inputs):.4f}, {np.max(f7_inputs):.4f}]")
print(f"Performance range: [{np.min(f7_outputs):.4f}, {np.max(f7_outputs):.4f}]")

The improvement is more significant this week. We can have another go at the same strategy once again.

In [ ]:
kernel = (
    C(1.0, (1e-2, 1e2)) * (
        RBF([1.0]*6, length_scale_bounds=(1e-2, 15)) +
        Matern(length_scale=[1.0]*6, length_scale_bounds=(1e-2,250), nu=2.5)
    ) +
    WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-13, 1e-1))
)

gp = GaussianProcessRegressor(kernel=kernel,
                              alpha=1e-6,
                              normalize_y=True,
                              n_restarts_optimizer=20,
                              random_state=42)
gp.fit(f7_inputs, f7_outputs)
print(gp.kernel_)

def negative_ucb(x, gp, kappa):
    mu, sigma = gp.predict(x.reshape(1, -1), return_std=True)
    return -(mu[0] + kappa * sigma[0])


bounds = [(0, 1) for _ in range(6)]

n_candidates = 3000000  # Much more manageable
np.random.seed(42)
margin=0.01
X_candidates = np.random.uniform(
    low=margin, 
    high=1-margin, 
    size=(n_candidates, 6)
)

y_mean, y_std = gp.predict(X_candidates, return_std=True)

# UCB acquisition function
kappa = 0.5 
UCB = y_mean + kappa * y_std 

next_idx = np.argmax(UCB)
next_point = X_candidates[next_idx]

print(f"Current best: {np.max(f7_outputs):.4f}")
print(f"Inputs producing current best: {f7_inputs[np.argmax(f7_outputs)]}")
print("Next point to sample based on UCB:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 3: 0.015145-0.204820-0.115783-0.977408-0.415003-0.770445

### Week 4

In [ ]:
#Load points to inputs and outputs from previous week
function = 7
week = 3 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f7_inputs = np.vstack([f7_inputs, row['provided_input'].values[0]])
f7_outputs = np.append(f7_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f7_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f7_inputs.shape}")
print(f"Inputs range: [{np.min(f7_inputs):.4f}, {np.max(f7_inputs):.4f}]")
print(f"Performance range: [{np.min(f7_outputs):.4f}, {np.max(f7_outputs):.4f}]")

No improvement this week. I am choosing a more simplified kernel and a UCB acquisition function to further explore this high-dimensional space for the week. I am also simplifying the kernel and switching to Sobol sampling, which is supposed to cover better this high-dimensional space (https://en.wikipedia.org/wiki/Sobol_sequence). 

In [ ]:
kernel = C(1.0, (1e-2, 1e2)) * Matern(length_scale=[1.0]*6, length_scale_bounds=(1e-2, 300), nu=2.5) + \
         WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-13, 1e-1))

#kernel = (
#    C(1.0, (1e-3, 1e3)) *
#    (
#        RBF(length_scale=[1.0]*6, length_scale_bounds=(1e-2, 75)) +
#        Matern(length_scale=[1.0]*6, length_scale_bounds=(1e-2, 1000), nu=2.5)
#    ) +
#    WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-13, 1e-1))
#)

gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-6,
    normalize_y=True,
    n_restarts_optimizer=40,
    random_state=42
)

gp.fit(f7_inputs, f7_outputs)
print(gp.kernel_)

def negative_ucb(x, gp, kappa):
    mu, sigma = gp.predict(x.reshape(1, -1), return_std=True)
    return -(mu[0] + kappa * sigma[0])

dims = 6
bounds = [(0, 1) for _ in range(dims)]


#num_restarts = 5000
#results = []
#np.random.seed(42)

#for _ in range(num_restarts):
#    x0 = np.random.uniform(0, 1, dims)
#    result = minimize(negative_ucb, x0=np.random.uniform(0, 1, dims), 
#                 args=(gp, 1.2), bounds=bounds, method='L-BFGS-B')
#    if result.success:
#        results.append(result)
    
# Select best solution from restarts
#next_point_ucb = min(results, key=lambda r: r.fun).x

n_candidates = 500000  # efficient but large enough
margin = 0.01

sampler = qmc.Sobol(d=dims, scramble=True, seed=42)
X_candidates = sampler.random(n_candidates)
X_candidates = margin + (1 - 2 * margin) * X_candidates  # stay inside (margin, 1-margin)

# Predict GP mean and std for all candidates
y_mean, y_std = gp.predict(X_candidates, return_std=True)

# Dynamic exploration factor
# Example: if you've done t of 5 remaining steps, gradually reduce kappa
t = len(f7_inputs)  # number of samples so far
kappa = 1.5

# UCB acquisition
UCB = y_mean + kappa * y_std
next_idx = np.argmax(UCB)
next_point_ucb = X_candidates[next_idx]


print(f"Current best observed value: {np.max(f7_outputs):.4f}")
print("Inputs producing current best:", f7_inputs[np.argmax(f7_outputs)])
print("Next sample point (ucb):", next_point_ucb)
print("-".join(f"{x:.6f}" for x in next_point_ucb.flatten()))

Submission for week 4: 0.036332-0.920708-0.509829-0.218803-0.361421-0.824300

### Week 5

In [ ]:
#Load points to inputs and outputs from previous week
function = 7
week = 4 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f7_inputs = np.vstack([f7_inputs, row['provided_input'].values[0]])
f7_outputs = np.append(f7_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f7_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f7_inputs.shape}")
print(f"Inputs range: [{np.min(f7_inputs):.4f}, {np.max(f7_inputs):.4f}]")
print(f"Performance range: [{np.min(f7_outputs):.4f}, {np.max(f7_outputs):.4f}]")

In [ ]:
f7_inputs_outputs = np.hstack([f7_inputs, f7_outputs.reshape(f7_inputs.shape[0],1)])
#print(f4_inputs_outputs)

print(f"Correlation between inputs and with outputs (last column): \n{np.corrcoef(f7_inputs_outputs, rowvar=False)}")

1st, 5th and 6th dimension are the ones with the highest correlation with the outputs in absolute value.

From the given data, "good" input values are those that produce an output > 1.3.

For the sake of experimentation and application of techniques, I am going to train a SVM classifier on this 6D space to separate the "good" part from the "bad" part of this 6D hypercube.

In [ ]:
svm_with_kernel = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, C=100, probability=True))
])

f7_output_types = (f7_outputs >= 1.3).astype(np.int16)

svm_with_kernel.fit(f7_inputs, f7_output_types)

f7_outputs_preds = svm_with_kernel.predict(f7_inputs)
f7_outputs_preds_probs = svm_with_kernel.predict_proba(f7_inputs)[:, 1]

print(f7_outputs_preds)
print(f7_outputs_preds_probs)

The datasample is very small to evaluate the classifier, but here is the confusion matrix and accuracy score. Is it overfitting?

In [ ]:
print(f"Confusion matrix: \n{confusion_matrix(f7_output_types, f7_outputs_preds)}")
print(f"Accuracy: {accuracy_score(f7_output_types, f7_outputs_preds):.4f}")

Let's explore the 6D-dimensional hypercube and try to find a point with the highest probability of belonging to the "good" category while being the furthest away from all known points so far to encourage exploration and test this method.

In [ ]:
dims = 6
n_samples = 200_000
lambda_distance = 0.5 # to play with to weigh distance importance

np.random.seed(42)
samples = np.random.rand(n_samples, dims)

neighbors = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(f7_inputs)
distances, _ = neighbors.kneighbors(samples)
distances = distances.flatten()

f7_outputs_preds_probs = svm_with_kernel.predict_proba(samples)[:, 1]

score = f7_outputs_preds_probs.flatten() + lambda_distance * distances

best_idx = np.argmax(score)
best_point = samples[best_idx]
best_prob = f7_outputs_preds_probs[best_idx]
best_dist = distances[best_idx]

print(f"Probability of being good input: {best_prob:.4f}")
print(f"Distance from known points: {best_dist:.4f}")
print("\n")
print(f"Current best: {np.max(f7_outputs):.4f}")
print(f"Inputs producing current best: {f8_inputs[np.argmax(f7_outputs)]}")
print("Next point to sample based on distance and prediction:", best_point)
print("-".join(f"{x:.6f}" for x in best_point.flatten()))

Input for week 5: 0.020515-0.735827-0.008273-0.047345-0.254657-0.980806

### Week 6

In [ ]:
#Load points to inputs and outputs from previous week
function = 7
week = 5 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f7_inputs = np.vstack([f7_inputs, row['provided_input'].values[0]])
f7_outputs = np.append(f7_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f7_outputs):.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")
print('--')
print(f"Inputs shape: {f7_inputs.shape}")
print(f"Inputs range: [{np.min(f7_inputs):.4f}, {np.max(f7_inputs):.4f}]")
print(f"Performance range: [{np.min(f7_outputs):.4f}, {np.max(f7_outputs):.4f}]")

## Function 8

This function is a function with a 1D output and a 8D input. 

This is the description of the function: You’re optimising an eight-dimensional black-box function, where each of the eight input parameters affects the output, but the internal mechanics are unknown. 

Your objective is to find the parameter combination that maximises the function’s output, such as performance, efficiency or validation accuracy. Because the function is high-dimensional and likely complex, global optimisation is hard, so identifying strong local maxima is often a practical strategy.

For example, imagine you’re tuning an ML model with eight hyperparameters: learning rate, batch size, number of layers, dropout rate, regularisation strength, activation function (numerically encoded), optimiser type (encoded) and initial weight range. Each input set returns a single validation accuracy score between 0 and 1. Your goal is to maximise this score.

### Week 1


In [ ]:
print(f"Evaluations: {len(f8_inputs)}")
print(f"Inputs shape: {f8_inputs.shape}")
print(f"Inputs range: [{np.min(f8_inputs):.4f}, {np.max(f8_inputs):.4f}]")
print(f"Performance range: [{np.min(f8_outputs):.4f}, {np.max(f8_outputs):.4f}]")
print(f"Current best: {np.max(f8_outputs):.4f}")
print(f"Inputs producing current best: {f8_inputs[np.argmax(f8_outputs)]}")


In [ ]:
# It's another high-dimensional function with not much information about it. I will replicate the random sampling technique and kernels and acquisition from function 7.

kernel = (
    C(1.0, (1e-6, 1e6)) * (
        RBF([1.0]*8, length_scale_bounds=(1e-2, 1e25)) +
        Matern(length_scale=[1.0]*8, length_scale_bounds=(1e-2,1e25), nu=2.5)
    ) +
    WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-13, 1e-1))
)

gp = GaussianProcessRegressor(kernel=kernel,
                              alpha=1e-6,
                              normalize_y=True,
                              n_restarts_optimizer=20,
                              random_state=42)
gp.fit(f8_inputs, f8_outputs)

margin = 0.01  # avoid boundaries, previous function gave a 1 for one of the coordinates

# Random sampling approach to avoid memory issues
n_candidates = 1000000  # Much more manageable
np.random.seed(42)
X_candidates = np.random.uniform(
    low=margin, 
    high=1-margin, 
    size=(n_candidates, 8)
)

y_mean, y_std = gp.predict(X_candidates, return_std=True)

# UCB acquisition function
kappa = 2.5 
UCB = y_mean + kappa * y_std 

next_idx = np.argmax(UCB)
next_point = X_candidates[next_idx]

print(f"Current best: {np.max(f8_outputs):.4f}")
print(f"Inputs producing current best: {f8_inputs[np.argmax(f8_outputs)]}")
print("Next point to sample based on UCB:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 1: 0.053588-0.199773-0.065100-0.022307-0.890284-0.392617-0.053845-0.582385

It is noted that I am getting many warnings about length scale, which is a topic to address the next week.

### Week 2

In [ ]:
#Load points to inputs and outputs from previous week
function = 8
week = 1 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f8_inputs = np.vstack([f8_inputs, row['provided_input'].values[0]])
f8_outputs = np.append(f8_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f8_outputs):.4f}")
print(f"Previous best: {results_df[(results_df['week'] == week-1) & (results_df['function'] == function)]['known_best'].values[0]:.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")

In [ ]:
print(f"Correlation between inputs: \n{np.corrcoef(f8_inputs, rowvar=False)}\n")

print('Means:', f8_inputs.mean(axis=0))
print('Stds:', f8_inputs.std(axis=0))
print('Mins:', f8_inputs.min(axis=0))
print('Maxs:', f8_inputs.max(axis=0))
print('Std/Mean Ratios:', f8_inputs.std(axis=0) / (f8_inputs.mean(axis=0) + 1e-12))

It is encouraged to exploit local minima, which we will start doing considering the improvement through the acquisition function. For the moment I am keeping UCB with a smaller kappa.

In [ ]:
# It's another high-dimensional function with not much information about it. I will replicate the random sampling technique and kernels and acquisition from function 7.

kernel = (
    C(1.0, (1e-6, 1e6)) * (
        RBF([1.0]*8, length_scale_bounds=(1e-2, 1e2)) +
        Matern(length_scale=[1.0]*8, length_scale_bounds=(1e-2,1e2), nu=1.5)
    ) +
    WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-13, 1e-1))
)

gp = GaussianProcessRegressor(kernel=kernel,
                              alpha=1e-6,
                              normalize_y=True,
                              n_restarts_optimizer=20,
                              random_state=42)
gp.fit(f8_inputs, f8_outputs)
print(gp.kernel_)

margin = 0.01  # avoid boundaries, previous function gave a 1 for one of the coordinates

# Random sampling approach to avoid memory issues
n_candidates = 1000000  # Much more manageable
np.random.seed(42)
X_candidates = np.random.uniform(
    low=margin, 
    high=1-margin, 
    size=(n_candidates, 8)
)

y_mean, y_std = gp.predict(X_candidates, return_std=True)

# UCB acquisition function
kappa = 0.25
UCB = y_mean + kappa * y_std 

next_idx = np.argmax(UCB)
next_point = X_candidates[next_idx]

print(f"Current best: {np.max(f8_outputs):.4f}")
print(f"Inputs producing current best: {f8_inputs[np.argmax(f8_outputs)]}")
print("Next point to sample based on UCB:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 2: 0.076967-0.216041-0.229342-0.023042-0.977037-0.446917-0.188639-0.130387

### Week 3

In [ ]:
#Load points to inputs and outputs from previous week
function = 8
week = 2 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f8_inputs = np.vstack([f8_inputs, row['provided_input'].values[0]])
f8_outputs = np.append(f8_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f8_outputs):.4f}")
print(f"Previous best: {results_df[(results_df['week'] == week-1) & (results_df['function'] == function)]['known_best'].values[0]:.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")

This is really a marginal improvement over next week. 

In [ ]:
# Dropping the RBF kernel
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[1.0]*8, length_scale_bounds=(1e-2, 1e3), nu=2.5) \
         + WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-13, 1e-1))


gp = GaussianProcessRegressor(kernel=kernel,
                              alpha=1e-6,
                              normalize_y=True,
                              n_restarts_optimizer=20,
                              random_state=42)
gp.fit(f8_inputs, f8_outputs)
print(gp.kernel_)

margin = 0.01  # avoid boundaries, previous function gave a 1 for one of the coordinates

# Random sampling approach to avoid memory issues
n_candidates = 1000000  # Much more manageable
np.random.seed(42)
X_candidates = np.random.uniform(
    low=margin, 
    high=1-margin, 
    size=(n_candidates, 8)
)

y_mean, y_std = gp.predict(X_candidates, return_std=True)

# UCB acquisition function
kappa = 0.5
UCB = y_mean + kappa * y_std 

next_idx = np.argmax(UCB)
next_point = X_candidates[next_idx]

print(f"Current best: {np.max(f8_outputs):.4f}")
print(f"Inputs producing current best: {f8_inputs[np.argmax(f8_outputs)]}")
print("Next point to sample based on UCB:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Submission for week 3: 0.054303-0.023167-0.046826-0.146788-0.981370-0.541223-0.235202-0.585401

### Week 4

In [ ]:
#Load points to inputs and outputs from previous week
function = 8
week = 3 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f8_inputs = np.vstack([f8_inputs, row['provided_input'].values[0]])
f8_outputs = np.append(f8_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f8_outputs):.4f}")
print(f"Previous best: {results_df[(results_df['week'] == week-1) & (results_df['function'] == function)]['known_best'].values[0]:.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")

Although small, this was an improvement.

This week I am using the same technique without the RBF component to the kernel, but replacing the way of evaluating random points of the GP with a Sobol sampling technique.

In [ ]:
dims = 8

# Dropping the RBF kernel
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[1.0]*dims, length_scale_bounds=(1e-2, 1e3), nu=2.5) \
         + WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-13, 1e-1))


gp = GaussianProcessRegressor(kernel=kernel,
                              alpha=1e-6,
                              normalize_y=True,
                              n_restarts_optimizer=20,
                              random_state=42)
gp.fit(f8_inputs, f8_outputs)
print(gp.kernel_)

margin = 0.01  # avoid boundaries, previous function gave a 1 for one of the coordinates

# Random sampling approach to avoid memory issues
# n_candidates = 1000000  # Much more manageable

np.random.seed(42)
#X_candidates = np.random.uniform(
#   low=margin, 
#    high=1-margin, 
#   size=(n_candidates, 8)
#)

sampler = qmc.Sobol(d=dims, scramble=True, seed=42)
#X_candidates = sampler.random(n_candidates)
X_candidates = sampler.random_base2(m=20) # 2**20 samples
X_candidates = margin + (1 - 2 * margin) * X_candidates  # stay inside (margin, 1-margin)

y_mean, y_std = gp.predict(X_candidates, return_std=True)

# UCB acquisition function
kappa = 0.5
UCB = y_mean + kappa * y_std 

next_idx = np.argmax(UCB)
next_point = X_candidates[next_idx]

print(f"Current best: {np.max(f8_outputs):.4f}")
print(f"Inputs producing current best: {f8_inputs[np.argmax(f8_outputs)]}")
print("Next point to sample based on UCB:", next_point)
print("-".join(f"{x:.6f}" for x in next_point.flatten()))

Sample for week 4: 0.229166-0.133413-0.151014-0.018894-0.960755-0.438649-0.170407-0.911107

### Week 5

In [ ]:
#Load points to inputs and outputs from previous week
function = 8
week = 4 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f8_inputs = np.vstack([f8_inputs, row['provided_input'].values[0]])
f8_outputs = np.append(f8_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f8_outputs):.4f}")
print(f"Previous best: {results_df[(results_df['week'] == week-1) & (results_df['function'] == function)]['known_best'].values[0]:.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")

In [ ]:
f8_inputs_outputs = np.hstack([f8_inputs, f8_outputs.reshape(f8_inputs.shape[0],1)])
#print(f4_inputs_outputs)

print(f"Correlation between inputs and with outputs (last column): \n{np.corrcoef(f8_inputs_outputs, rowvar=False)}")


Outputs are most correlated with the first and third input, negatively.

Let's see if we can reduce the inputs to 4D.

In [ ]:
pca = PCA(n_components = 4)
f8_inputs_3d = pca.fit_transform(f8_inputs)

print(pca.explained_variance_ratio_)

Variance would only be explained up to 0.68.

The function description encourages the exploration of local minima, which we seem to have found with an output value > 9.

For experimentation purposes I want to train a neural network based on these 8 dimensions to determine whether the inputs produce "good" outputs (>8.9). Then I can allow myself to perform a random search accross the 8D space to find other points with good outputs as far as possible from known points. This may allow me to find another area with local minima to explore which may even yield higher outpupt values. Can be fun!

In [ ]:
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Sequential model construction
model = Sequential([
    layers.Dense(8, activation="relu"),
    layers.Dense(4, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

x = tf.convert_to_tensor(f8_inputs.astype(np.float32))
y = (f8_outputs >= 8.5)
y = tf.convert_to_tensor(y.astype(np.int16))

model.compile(optimizer="adam", metrics=["accuracy"], loss="binary_crossentropy")

history = model.fit(x,y, epochs=550, validation_split=0.15)
y_pred = model.predict(x)

# (Optional) Visualize the computational graph
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)
# Display in Jupyter Notebook
model.summary()


In [ ]:
# Let's try to find the point with the highest probability of having a 1 (good value) acording to the NN that is furthest away from known points.

dims = 8
n_samples = 100_000
lambda_distance = 0.5 # to play with to weigh distance importance

np.random.seed(42)
samples = np.random.rand(n_samples, dims)

probs_of_good_result = model.predict(samples, verbose=0).flatten()

neighbors = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(f8_inputs)
distances, _ = neighbors.kneighbors(samples)
distances = distances.flatten()

score = probs_of_good_result + lambda_distance * distances

best_idx = np.argmax(score)
best_point = samples[best_idx]
best_prob = probs_of_good_result[best_idx]
best_dist = distances[best_idx]

print(f"Probability of being good input: {best_prob:.4f}")
print(f"Distance from known points: {best_dist:.4f}")
print("\n")
print(f"Current best: {np.max(f8_outputs):.4f}")
print(f"Inputs producing current best: {f8_inputs[np.argmax(f8_outputs)]}")
print("Next point to sample based on distance and prediction:", best_point)
print("-".join(f"{x:.6f}" for x in best_point.flatten()))

Submission for week 5: 0.066684-0.029599-0.012412-0.855233-0.692224-0.676637-0.760723-0.973119

*I was unable to fix the random seed for TF this week, so probably next runs will yield different points. However the one listed here is the one submitted as part of this capstone project.

Let's see if the NN has learned the underlying patterns!

### Week 6

In [ ]:
#Load points to inputs and outputs from previous week
function = 8
week = 5 #previous

row = results_df[(results_df['week'] == week) & (results_df['function'] == function)]
f8_inputs = np.vstack([f8_inputs, row['provided_input'].values[0]])
f8_outputs = np.append(f8_outputs, row['output'].values[0])

improved = row['submission_improved'].values[0]
print(f"Submission from last week improved something? {improved}")
print(f"Current best: {np.max(f8_outputs):.4f}")
print(f"Previous best: {results_df[(results_df['week'] == week-1) & (results_df['function'] == function)]['known_best'].values[0]:.4f}")
print(f"Last week's results: {row['output'].values[0]:.4f}")